# Code to map velocities using autoRIFT with adjustable parameters

_Last modified on 2022-05-17 by jukesliu@u.boisestate.edu._

Majority of the code is borrowed from testGeogrid_ISCE.py and testautoRIFT_ISCE.py from the autoRIFT GitHub repository (https://github.com/nasa-jpl/autoRIFT/). Requires ISCE and autoRIFT to be installed.

Change the kernel to __newautoriftenv__.

In [1]:
import rasterio as rio
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from scipy.interpolate import interp2d

import isce
from contrib.geo_autoRIFT.geogrid import Geogrid, GeogridOptical
from iscesys.Component.ProductManager import ProductManager as PM
from isceobj.Orbit.Orbit import Orbit
from osgeo import gdal, osr
import struct
import re
from datetime import date
import datetime
import xml.etree.ElementTree as ET
import urllib.request
from components.contrib.geo_autoRIFT.autoRIFT import autoRIFT_ISCE
import isceobj
import time
import subprocess
import cv2
import scipy.io as sio

gdal.AllRegister() # register all GDAL drivers

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce


## 1) Resample DEM and other autoRIFT/geogrid input rasters to the desired chip size

Recommended chip size is >= 16*pixel_resolution

#### Customizable parameters for geogrid:

    dhdx, dhdy:              x/y local surface slope maps (unitless)
    vx,vy:                   x/y reference velocity maps (in units of m/yr)
    srx, sry:                x/y velocity search range limit maps (in units of m/yr)
    csminx, csminy:          x/y chip size minimum maps (in units of m; constant ratio between x and y)
    csmaxx, csmaxy:          x/y chip size maximum maps (in units of m; constant ratio between x and y)
    ssm:                     stable surface mask (boolean)

In [2]:
def rio_write(out_path, nparray, ref_raster, grid_spacing):
    # Function to write a numpy array to a Geotiff using rasterio
    # The geotiff will have the same bounds and crs as the reference raster
    # An evenly-spaced grid will be created with the grid spacing entered
    #
    # INPUTS:
    #   out_path: the path with name of the output gtiff file
    #   nparray: the nparray to write to gtiff
    #   ref_raster: the reference raster (we borrow its crs and bounding coordinates)
    #   grid_spacing: the spatial resolution of the output raster
    
    import rasterio as rio
    import numpy as np
    nparray = np.array(nparray) # make sure it's an np array
    
    with rio.open(out_path,'w',
                      driver='GTiff',
                      height=nparray.shape[0], # new shape
                      width=nparray.shape[1], # new shape
                      dtype=nparray.dtype, # data type
                      count=1,
                      crs=ref_raster.crs, # the EPSG from the original DEM
                      transform=rio.Affine(grid_spacing, 0.0, ref_raster.bounds.left, # modified transform
                                           0.0, -grid_spacing, ref_raster.bounds.top)) as dst:
            dst.write(nparray, 1)

In [3]:
######### ENTER CHIP SIZE, DEM INFO, AND REFERENCE VELOCITY INFO ########### 
CHIPSIZE_M = 100 # enter in desired grid size in meters (default is 32 pixels)

# enter in the path to your best DEM over the region
dempath = '/Users/jukesliu/Documents/TURNER/DATA/ICE_THICKNESS/surface/DEMs_previous/'
demname = 'IfSAR_5m_DSM_clipped.tif'

# path to the reference files for geogrid (vx, vy, ssm)
refvpath = '/Users/jukesliu/Documents/TURNER/DATA/VELOCITY_MAPS/forAutoRIFT/' 
vx_fname = 'vx_cropped.tif' # name of reference vx file
vy_fname = 'vy_cropped.tif' # name of reference vy file

sr_scaling = 16 # multiply by vx and vy to generate search range limits
############################################################################

In [4]:
# ONE CHIPSIZE:
generate_geogrid_inputs(CHIPSIZE_M, dempath, demname, refvpath, vx_fname, vy_fname, sr_scaling)

# # MANY CHIPSIZES:
# for CHIPSIZE_M in [100,150,160,200,300, 350, 400]:
#     generate_geogrid_inputs(CHIPSIZE_M, dempath, demname, refvpath, vx_fname, vy_fname, sr_scaling)
#     print(); print()

NameError: name 'generate_geogrid_inputs' is not defined

In [6]:
def generate_geogrid_inputs(CHIPSIZE_M, dempath, demname, refvpath, vx_fname, vy_fname, sr_scaling):
    
    # GRAB DEM INFO
    refdem = rio.open(dempath+demname) # open DEM using rasterio
    elev = refdem.read(1) # read in the first and only band (elevations)

    # grab the x and y grid values from the DEM:
    dem_x = np.linspace(refdem.bounds.left, refdem.bounds.right, num=np.shape(elev)[1])
    dem_y = np.linspace(refdem.bounds.top, refdem.bounds.bottom, num=np.shape(elev)[0])

    # grab the resampled x and y grid values from the DEM
    new_x = np.arange(refdem.bounds.left, refdem.bounds.right, CHIPSIZE_M)
    new_y = np.arange(refdem.bounds.top, refdem.bounds.bottom, -CHIPSIZE_M)
    
    # RESAMPLE THE DEM
    dem_outfile = 'IfSAR_'+str(CHIPSIZE_M)+'m_DSM_clipped.tif' # generate new filename
    if not os.path.exists(dempath+dem_outfile): # if the resampled DEM does not exist already
        # Create thew new x and y grid values using DEM bounds and the chipsize
        dem_resamp = np.zeros((len(new_y), len(new_x))) # create an empty resampled DEM grid
        print(dem_resamp.shape)

        # Resample to your new DEM bounds
        f = interp2d(dem_x, dem_y, elev) # create DEM interpolation object
        dem_resamp = f(new_x,new_y) # resample the NIR data to the DSM coordinates
        dem_resamp = np.flipud(dem_resamp) # flip up down
        print("Resampled to new dimensions:",dem_resamp.shape)

        # Display the two DEMs as a visual check
        fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,5))
        im1 = ax1.imshow(elev, cmap='Greys_r', vmin=0)
        ax1.set_title('Original DEM: '+str(refdem.transform[0])+' m') # original spatial resolution
        fig.colorbar(im1, ax=ax1,label='Elevation [m]')

        im2 = ax2.imshow(dem_resamp, cmap='Greys_r', vmin=0)
        ax2.set_title('Resampled DEM: '+str(CHIPSIZE_M)+' m') # new spatial resolution
        fig.colorbar(im2, ax=ax2,label='Elevation [m]')
        plt.show()

        # Save the resampled DEM to georeferenced tif file
        print("Save resampled DEM to", dempath+dem_outfile)
        rio_write(dempath+dem_outfile, dem_resamp, refdem, CHIPSIZE_M)
    else:
        # load the existing resampled DEM
        dem_r = rio.open(dempath+dem_outfile) # open DEM using rasterio
        dem_resamp = dem_r.read(1) # read in the first and only band (elevations)
        print(dem_outfile, ' already exists.')
    
    # CREATE DHDX, DHDY
    dhdx_outfile = 'IfSAR_'+str(CHIPSIZE_M)+'m_DSM_clipped_dhdx.tif' # generate new filename
    dhdy_outfile = 'IfSAR_'+str(CHIPSIZE_M)+'m_DSM_clipped_dhdy.tif' # generate new filename
    if not os.path.exists(dempath+dhdx_outfile) or not os.path.exists(dempath+dhdy_outfile): # if either is missing
        # Produce dhdx and dhdy maps from resampled DEM
        dhdx = np.gradient(dem_resamp, axis=1)/CHIPSIZE_M
        dhdy = np.gradient(dem_resamp, axis=0)/CHIPSIZE_M

        # Filter out borders with high gradient values
        grad_thresh = 5
        dhdx[abs(dhdx) > grad_thresh] = 0; dhdy[abs(dhdy) > grad_thresh] = 0

        # absolute value of the max gradient values expected:
        dhmax = 1

        # Display the two DEMs as a visual check
        fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,5))
        im1 = ax1.imshow(dhdx, cmap='Greys_r', vmin=-dhmax, vmax=dhmax)
        ax1.set_title('dhdx') # surface slope x
        fig.colorbar(im1, ax=ax1)

        im2 = ax2.imshow(dhdy, cmap='Greys_r', vmin=-dhmax, vmax=dhmax)
        ax2.set_title('dhdy') # surface slope y
        fig.colorbar(im2, ax=ax2)
        plt.show()

        # Save the gradient maps to tif files
        print("Save surface slope maps to", dempath)
        rio_write(dempath+dhdx_outfile, dhdx, refdem, CHIPSIZE_M) # dhdx
        rio_write(dempath+dhdy_outfile, dhdy, refdem, CHIPSIZE_M)
    else:
        print(dhdy_outfile, 'and', dhdx_outfile, 'already exist.')

    # VX, VY, SRX, SRY
    vx_outfile = 'vx_'+str(CHIPSIZE_M)+'m.tif' # generate new filename
    vy_outfile = 'vy_'+str(CHIPSIZE_M)+'m.tif' # generate new filename
    srx_outfile = 'srx_'+str(CHIPSIZE_M)+'m.tif' # generate new filename
    sry_outfile = 'sry_'+str(CHIPSIZE_M)+'m.tif' # generate new filename
    if not os.path.exists(refvpath+vx_outfile) or not os.path.exists(refvpath+vy_outfile): # if either vx, vy missing
        # open the files with rasterio
        vx_reader = rio.open(refvpath+vx_fname); vx0 = vx_reader.read(1)
        vy_reader = rio.open(refvpath+vy_fname); vy0 = vy_reader.read(1)
        vx_x = np.linspace(vx_reader.bounds.left, vx_reader.bounds.right, num=np.shape(vx0)[1])
        vx_y = np.linspace(vx_reader.bounds.top, vx_reader.bounds.bottom, num=np.shape(vx0)[0])
        vy_x = np.linspace(vy_reader.bounds.left, vy_reader.bounds.right, num=np.shape(vy0)[1])
        vy_y = np.linspace(vy_reader.bounds.top, vy_reader.bounds.bottom, num=np.shape(vy0)[0])

        # Resample to the DEM grid
        fx = interp2d(vx_x, vx_y, vx0)
        fy = interp2d(vy_x, vy_y, vy0)
        vx_resamp = np.flipud(fx(new_x,new_y)) 
        vy_resamp = np.flipud(fy(new_x,new_y)) # flip up down

        # Display the two velocity files as a visual check
        fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,5))
        im1 = ax1.imshow(vx_resamp, cmap='Greys_r'); ax1.set_title('vx'); fig.colorbar(im1, ax=ax1)
        im2 = ax2.imshow(vy_resamp, cmap='Greys_r'); ax2.set_title('vy'); fig.colorbar(im2, ax=ax2)
        plt.show()

        # CALCULATE SEARCH RANGE LIMITS MULTIPLY VX AND VY BY SOME NUMBER
        srx_resamp = vx_resamp*sr_scaling; sry_resamp = vy_resamp*sr_scaling

        # Display the two search range files as a visual check
        fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,5))
        im1 = ax1.imshow(srx_resamp, cmap='Greys_r'); ax1.set_title('srx'); fig.colorbar(im1, ax=ax1)
        im2 = ax2.imshow(sry_resamp, cmap='Greys_r'); ax2.set_title('sry'); fig.colorbar(im2, ax=ax2)
        plt.show()

        # save the reference velocity and search range maps
        rio_write(refvpath+vx_outfile, vx_resamp, refdem, CHIPSIZE_M) # vx
        rio_write(refvpath+vy_outfile, vy_resamp, refdem, CHIPSIZE_M) # vy
        rio_write(refvpath+srx_outfile, srx_resamp, refdem, CHIPSIZE_M) # srx
        rio_write(refvpath+sry_outfile, sry_resamp, refdem, CHIPSIZE_M) # sry
    else:
        print(vx_outfile, ',', vy_outfile, ',', srx_outfile, ',', sry_outfile, 'already exist.')  
    
    # MASKS
    ssm_outfile = 'ssm_'+str(CHIPSIZE_M)+'m.tif' # generate new filename
    tg_outfile = 'TG_mask_'+str(CHIPSIZE_M)+'m.tif' # generate new filename
    tg_mt_outfile = 'TG_mask_MT_'+str(CHIPSIZE_M)+'m.tif'
    tg_nt_outfile = 'TG_mask_NT_'+str(CHIPSIZE_M)+'m.tif'
    tg_st_outfile = 'TG_mask_ST_'+str(CHIPSIZE_M)+'m.tif'

    if True: # overwrite all
        # read it in, process (resample, mask, etc.) and resave
        ssmreader = rio.open(refvpath+'ice_mask_200mbuffer.tif')
        ssm = ssmreader.read(1)
        ssm[ssm > 0] = 1; #ssm[ssm < 0.0] = 0; # make binary
        ssm = ssm < 1 # find all stable areas (where.tif = 0)

        # do the same for Turner Glacier mask
        tgreader = rio.open(refvpath+'TG_mask.tif'); tg_mask = tgreader.read(1)
        tg_mask[tg_mask > 0] = 1; tg_mask = tg_mask > 0
        
        # and all the regional masks
        tg_mt_reader = rio.open(refvpath+'TG_mask_MT.tif'); tg_mt = tg_mt_reader.read(1)
        tg_mt[tg_mt > 0] = 1; tg_mt = tg_mt > 0
        tg_nt_reader = rio.open(refvpath+'TG_mask_NT.tif'); tg_nt = tg_nt_reader.read(1)
        tg_nt[tg_nt > 0] = 1; tg_nt = tg_nt > 0
        tg_st_reader = rio.open(refvpath+'TG_mask_ST.tif'); tg_st = tg_st_reader.read(1)
        tg_st[tg_st > 0] = 1; tg_st = tg_st > 0
        
#         fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(10,5)); ax1.imshow(tg_mt); ax1.set_title('main truk')
#         ax2.imshow(tg_nt); ax2.set_title('N tributary'); ax3.imshow(tg_st); ax3.set_title('S tributary')
#         plt.show()

        # grab x and y-values
        ssm_x = np.linspace(ssmreader.bounds.left, ssmreader.bounds.right, num=np.shape(ssm)[1])
        ssm_y = np.linspace(ssmreader.bounds.top, ssmreader.bounds.bottom, num=np.shape(ssm)[0])
        tg_x = np.linspace(tgreader.bounds.left, tgreader.bounds.right, num=np.shape(tg_mask)[1])
        tg_y = np.linspace(tgreader.bounds.top, tgreader.bounds.bottom, num=np.shape(tg_mask)[0])
        tg_mtx = np.linspace(tg_mt_reader.bounds.left, tg_mt_reader.bounds.right, num=np.shape(tg_mt)[1])
        tg_mty = np.linspace(tg_mt_reader.bounds.top, tg_mt_reader.bounds.bottom, num=np.shape(tg_mt)[0])

        # Resample to the DEM grid
        f_ssm = interp2d(ssm_x, ssm_y, ssm)
        f_tg = interp2d(tg_x, tg_y, tg_mask)
        f_tg_mt = interp2d(tg_mtx, tg_mty, tg_mt)
        f_tg_nt = interp2d(tg_mtx, tg_mty, tg_nt)
        f_tg_st = interp2d(tg_mtx, tg_mty, tg_st)
        
        ssm_resamp = np.flipud(f_ssm(new_x,new_y))
        tg_resamp = np.flipud(f_tg(new_x, new_y))
        tg_mt_resamp = np.flipud(f_tg_mt(new_x, new_y))
        tg_nt_resamp = np.flipud(f_tg_nt(new_x, new_y))
        tg_st_resamp = np.flipud(f_tg_st(new_x, new_y))
        
        # plot
        fig, ax = plt.subplots(1,1)
        ssm_im = ax.imshow(ssm_resamp,cmap='gray',vmin=0)
        ax.set_title('Stable Surface Mask')
        fig.colorbar(ssm_im, ax=ax)
        plt.show()

        # export
        rio_write(refvpath+ssm_outfile, ssm_resamp, refdem, CHIPSIZE_M)
        rio_write(refvpath+tg_outfile, tg_resamp, refdem, CHIPSIZE_M)
        rio_write(refvpath+tg_mt_outfile, tg_mt_resamp, refdem, CHIPSIZE_M)
        rio_write(refvpath+tg_nt_outfile, tg_nt_resamp, refdem, CHIPSIZE_M)
        rio_write(refvpath+tg_st_outfile, tg_st_resamp, refdem, CHIPSIZE_M)
    else:
        print(ssm_outfile,'and',tg_outfile,'already exist.')
    
    return dem_outfile, dhdx_outfile, dhdy_outfile, vx_outfile, vy_outfile, srx_outfile, sry_outfile, ssm_outfile, tg_outfile
        

In [7]:
# Read and grab info from your DEM
refdem = rio.open(dempath+demname) # open DEM using rasterio
elev = refdem.read(1) # read in the first and only band (elevations)

# grab the x and y grid values from the DEM:
dem_x = np.linspace(refdem.bounds.left, refdem.bounds.right, num=np.shape(elev)[1])
dem_y = np.linspace(refdem.bounds.top, refdem.bounds.bottom, num=np.shape(elev)[0])

# grab the resampled x and y grid values from the DEM
new_x = np.arange(refdem.bounds.left, refdem.bounds.right, CHIPSIZE_M)
new_y = np.arange(refdem.bounds.top, refdem.bounds.bottom, -CHIPSIZE_M)

In [8]:
dem_outfile = 'IfSAR_'+str(CHIPSIZE_M)+'m_DSM_clipped.tif' # generate new filename

if not os.path.exists(dempath+dem_outfile): # if the resampled DEM does not exist already
    # Create thew new x and y grid values using DEM bounds and the chipsize
    dem_resamp = np.zeros((len(new_y), len(new_x))) # create an empty resampled DEM grid
    print(dem_resamp.shape)
    
    # Resample to your new DEM bounds
    f = interp2d(dem_x, dem_y, elev) # create DEM interpolation object
    dem_resamp = f(new_x,new_y) # resample the NIR data to the DSM coordinates
    dem_resamp = np.flipud(dem_resamp) # flip up down
    print("Resampled to new dimensions:",dem_resamp.shape)
    
    # Display the two DEMs as a visual check
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,5))
    im1 = ax1.imshow(elev, cmap='Greys_r', vmin=0)
    ax1.set_title('Original DEM: '+str(refdem.transform[0])+' m') # original spatial resolution
    fig.colorbar(im1, ax=ax1,label='Elevation [m]')

    im2 = ax2.imshow(dem_resamp, cmap='Greys_r', vmin=0)
    ax2.set_title('Resampled DEM: '+str(CHIPSIZE_M)+' m') # new spatial resolution
    fig.colorbar(im2, ax=ax2,label='Elevation [m]')
    plt.show()
    
    # Save the resampled DEM to georeferenced tif file
    print("Save resampled DEM to", dempath+dem_outfile)
    rio_write(dempath+dem_outfile, dem_resamp, refdem, CHIPSIZE_M)
else:
    # load the empty grid
    dem_resamp = np.zeros((len(new_y), len(new_x))) # create an empty resampled DEM grid
    print(dem_outfile, ' already exists.')

IfSAR_100m_DSM_clipped.tif  already exists.


## Generate geogrid outputs one at a time

### dhdx, dhdy

In [17]:
dhdx_outfile = 'IfSAR_'+str(CHIPSIZE_M)+'m_DSM_clipped_dhdx.tif' # generate new filename
dhdy_outfile = 'IfSAR_'+str(CHIPSIZE_M)+'m_DSM_clipped_dhdy.tif' # generate new filename

if not os.path.exists(dempath+dhdx_outfile) or not os.path.exists(dempath+dhdy_outfile): # if either is missing
    # Produce dhdx and dhdy maps from resampled DEM
    dhdx = np.gradient(dem_resamp, axis=1)/CHIPSIZE_M
    dhdy = np.gradient(dem_resamp, axis=0)/CHIPSIZE_M

    # Filter out borders with high gradient values
    grad_thresh = 5
    dhdx[abs(dhdx) > grad_thresh] = 0; dhdy[abs(dhdy) > grad_thresh] = 0

    # absolute value of the max gradient values expected:
    dhmax = 1

    # Display the two DEMs as a visual check
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,5))
    im1 = ax1.imshow(dhdx, cmap='Greys_r', vmin=-dhmax, vmax=dhmax)
    ax1.set_title('dhdx') # surface slope x
    fig.colorbar(im1, ax=ax1)

    im2 = ax2.imshow(dhdy, cmap='Greys_r', vmin=-dhmax, vmax=dhmax)
    ax2.set_title('dhdy') # surface slope y
    fig.colorbar(im2, ax=ax2)
    plt.show()
    
    # Save the gradient maps to tif files
    print("Save surface slope maps to", dempath)
    rio_write(dempath+dhdx_outfile, dhdx, refdem, CHIPSIZE_M) # dhdx
    rio_write(dempath_dhdy_outfile, dhdy, refdem, CHIPSIZE_M)
else:
    print(dhdy_outfile, 'and', dhdx_outfile, 'already exist.')

IfSAR_200m_DSM_clipped_dhdy.tif and IfSAR_200m_DSM_clipped_dhdx.tif already exist.


### vx, vy, srx, sry

In [18]:
############### SEARCH RANGE MULTIPLICATION FACTOR ###############
# search range limits are calculated as a scalar multiplied by the
# reference velocity distribution
sr_scaling = 16
##################################################################

# generate outfile names
vx_outfile = 'vx_'+str(CHIPSIZE_M)+'m.tif' # generate new filename
vy_outfile = 'vy_'+str(CHIPSIZE_M)+'m.tif' # generate new filename
srx_outfile = 'srx_'+str(CHIPSIZE_M)+'m.tif' # generate new filename
sry_outfile = 'sry_'+str(CHIPSIZE_M)+'m.tif' # generate new filename

if not os.path.exists(refvpath+vx_outfile) or not os.path.exists(refvpath+vy_outfile): # if either vx, vy missing
    # open the files with rasterio
    vx_reader = rio.open(refvpath+vx_fname); vx0 = vx_reader.read(1)
    vy_reader = rio.open(refvpath+vy_fname); vy0 = vy_reader.read(1)
    vx_x = np.linspace(vx_reader.bounds.left, vx_reader.bounds.right, num=np.shape(vx0)[1])
    vx_y = np.linspace(vx_reader.bounds.top, vx_reader.bounds.bottom, num=np.shape(vx0)[0])
    vy_x = np.linspace(vy_reader.bounds.left, vy_reader.bounds.right, num=np.shape(vy0)[1])
    vy_y = np.linspace(vy_reader.bounds.top, vy_reader.bounds.bottom, num=np.shape(vy0)[0])

    # Resample to the DEM grid
    fx = interp2d(vx_x, vx_y, vx0)
    fy = interp2d(vy_x, vy_y, vy0)
    vx_resamp = np.flipud(fx(new_x,new_y)) 
    vy_resamp = np.flipud(fy(new_x,new_y)) # flip up down

    # Display the two velocity files as a visual check
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,5))
    im1 = ax1.imshow(vx_resamp, cmap='Greys_r'); ax1.set_title('vx'); fig.colorbar(im1, ax=ax1)
    im2 = ax2.imshow(vy_resamp, cmap='Greys_r'); ax2.set_title('vy'); fig.colorbar(im2, ax=ax2)
    plt.show()

    # CALCULATE SEARCH RANGE LIMITS MULTIPLY VX AND VY BY SOME NUMBER
    srx_resamp = vx_resamp*sr_scaling; sry_resamp = vy_resamp*sr_scaling

    # Display the two search range files as a visual check
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,5))
    im1 = ax1.imshow(srx_resamp, cmap='Greys_r'); ax1.set_title('srx'); fig.colorbar(im1, ax=ax1)
    im2 = ax2.imshow(sry_resamp, cmap='Greys_r'); ax2.set_title('sry'); fig.colorbar(im2, ax=ax2)
    plt.show()
    
    # save the reference velocity and search range maps
    rio_write(refvpath+vx_outfile, vx_resamp, refdem, CHIPSIZE_M) # vx
    rio_write(refvpath+vy_outfile, vy_resamp, refdem, CHIPSIZE_M) # vy
    rio_write(refvpath+srx_outfile, srx_resamp, refdem, CHIPSIZE_M) # srx
    rio_write(refvpath+sry_outfile, sry_resamp, refdem, CHIPSIZE_M) # sry
else:
    print(vx_outfile, ',', vy_outfile, ',', srx_outfile, ',', sry_outfile, 'already exist.')
    
#     # OPTIONAL REMOVE
#     os.remove(refvpath+vx_outfile); os.remove(refvpath+vy_outfile); 
#     os.remove(refvpath+srx_outfile); os.remove(refvpath+sry_outfile);

vx_200m.tif , vy_200m.tif , srx_200m.tif , sry_200m.tif already exist.


### stable surface mask - optional

In [19]:
CHIPSIZE_M = 200

In [20]:
ssm_outfile = 'ssm_'+str(CHIPSIZE_M)+'m.tif' # generate new filename
tg_outfile = 'TG_mask_'+str(CHIPSIZE_M)+'m.tif' # generate new filename

if not os.path.exists(refvpath+ssm_outfile) or not os.path.exists(refvpath+tg_outfile):
    # read it in, process (resample, mask, etc.) and resave
    ssmreader = rio.open(refvpath+'ice_mask.tif')
    ssm = ssmreader.read(1)
    ssm[ssm > 0] = 1; #ssm[ssm < 0.0] = 0; # make binary
    ssm = ssm < 0 # find all stable areas (where.tif = 0)
    
    # do the same for Turner Glacier mask
    tgreader = rio.open(refvpath+'TG_mask.tif'); tg_mask = tgreader.read(1)
    tg_mask[tg_mask > 0] = 1
#     tg_mask = tg_mask < 0

    # grab x and y-values
    ssm_x = np.linspace(ssmreader.bounds.left, ssmreader.bounds.right, num=np.shape(ssm)[1])
    ssm_y = np.linspace(ssmreader.bounds.top, ssmreader.bounds.bottom, num=np.shape(ssm)[0])
    tg_x = np.linspace(tgreader.bounds.left, tgreader.bounds.right, num=np.shape(tg_mask)[1])
    tg_y = np.linspace(tgreader.bounds.top, tgreader.bounds.bottom, num=np.shape(tg_mask)[0])

    # Resample to the DEM grid
    f_ssm = interp2d(ssm_x, ssm_y, ssm)
    f_tg = interp2d(tg_x, tg_y, tg_mask)
    ssm_resamp = np.flipud(f_ssm(new_x,new_y))
    tg_resamp = np.flipud(f_tg(new_x, new_y))

    # plot
    fig, ax = plt.subplots(1,1)
    ssm_im = ax.imshow(tg_resamp,cmap='gray',vmin=0)
    ax.set_title('Stable Surface Mask')
    fig.colorbar(ssm_im, ax=ax)
    plt.show()

#     # export
#     rio_write(refvpath+ssm_outfile, ssm_resamp, refdem, CHIPSIZE_M)
    rio_write(refvpath+tg_outfile, tg_resamp, refdem, CHIPSIZE_M)
else:
    print(ssm_outfile,'and',tg_outfile,'already exist.')

ssm_200m.tif and TG_mask_200m.tif already exist.


### csminx, csminy, csmax, csmaxy - optional

In [21]:
# generate the file names
csminx_fname = 'csminx_'+str(CHIPSIZE_M)+'m.tif'
csminy_fname = 'csminy_'+str(CHIPSIZE_M)+'m.tif'
csmaxx_fname = 'csmaxx_'+str(CHIPSIZE_M)+'m.tif'
csmaxy_fname = 'csmaxy_'+str(CHIPSIZE_M)+'m.tif'

if not os.path.exists(refvpath+csminx_fname) or not os.path.exists(refvpath+csminy_fname): # if either csminx or csminy missing:
    # create the rasters
    csminx = np.ones(dem_resamp.shape)*CHIPSIZE_M # minimum chip size shaped like the resampled DEM
    csminy = csminx # make identical in both directions
    csmaxx = csminx*4 # make maximum chip size 4*minimum chip size (400)
    csmaxy = csmaxx
    
    # export
    rio_write(refvpath+csminx_fname, csminx, refdem, CHIPSIZE_M) # csminx
    rio_write(refvpath+csminy_fname, csminy, refdem, CHIPSIZE_M) # csminy
    rio_write(refvpath+csmaxx_fname, csmaxx, refdem, CHIPSIZE_M) # csmaxx
    rio_write(refvpath+csmaxy_fname, csmaxy, refdem, CHIPSIZE_M) # csmaxx
else:
    print(csminx_fname, ',',csminy_fname, ',', csmaxx_fname, ',', csmaxy_fname, 'already exist.')

csminx_200m.tif , csminy_200m.tif , csmaxx_200m.tif , csmaxy_200m.tif already exist.


## 2) Run geogrid with resampled DEM and other inputs

In [22]:
###### SET PATH PROCESSING PATHS ##########################
out_path = '/Users/jukesliu/Documents/TURNER/DATA/VELOCITY_MAPS/output_AutoRIFT/' # output file path
# # LS test:
# imgpath = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/LS8images/useable_images/'
# im1_name = 'LC08_L1TP_062018_20200401_20200410_01_T1_B8_BufferTurner.TIF'
# im2_name = 'LC08_L1TP_062018_20200417_20200423_01_T1_B8_BufferTurner.TIF'
# img_type = 'OPT'

# # S2 test:
# imgpath = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel2/'
# im1_name = 'S2A_7VEG_20200427_B08_clipped.tif'
# im2_name = 'S2A_7VEG_20200510_B08_clipped.tif'
# img_type = 'OPT'

# # PS test:
# imgpath = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/Planet_test/'
# im1_name = 'PS_20190304_clipped.tif'
# im2_name = 'PS_20190307_clipped.tif'
# img_type = 'OPT'

# S1 test
imgpath = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel1/'
im1_name = 'S1A_IW_SLC__1SDV_20210322T154835_20210322T154901_037113_045E94_6840.zip'
im2_name = 'S1A_IW_SLC__1SDV_20210403T154835_20210403T154902_037288_04649A_5AF4.zip'
img_type = 'SAR'
if not os.path.exists(imgpath+'topsApp.py'):
    print('topsApp.py not in folder. Place topsApp.py in', imgpath)
else:
    print('topsApp.py available for SAR image coregistration.')

#########################################################

topsApp.py available for SAR image coregistration.


In [14]:
# def run_geogrid_inhouse(out_path, img_type, indir_m, indir_s, MINCHIPSIZE, NO_DATA_VAL, dem, # required inputs
#                         dhdx, dhdy, vx, vy, srx, sry, csminx, csminy, csmaxx, csmaxy, ssm, # optional inputs
#                        temp_dir): # SAR needed only
    
#     CHIPSIZE_M = MINCHIPSIZE # set minimum chip size equal
#     ############ Clear all old geogrid files ##########################
#     for file in os.listdir(out_path):
#         if file.startswith('window') and file.endswith('.tif'):
#             print('removed', file)
#             os.remove(out_path+file)
#     print('Old files cleared.'); print()

#     dem_info = gdal.Info(dem, format='json') # grab info from DEM
#     print('Obtained DEM info.'); print()

#     ############ Run geogrid optical or SAR ##########################
#     if img_type == 'OPT': # Optical images
#         print('Processing optical images with geogrid.'); print()
#         obj = GeogridOptical() # initialize geogrid object

#         ############ Coregister the optical data (from coregisterLoadMetadataOptical) #############
#         x1a, y1a, xsize1, ysize1, x2a, y2a, xsize2, ysize2, trans = obj.coregister(indir_m, indir_s,0)

#         # grab dates from file names
#         im1_name = indir_m.split('/')[-1]; im2_name = indir_s.split('/')[-1]
#         if 'LC' in im1_name and 'LC' in im2_name:
#             ds1 = im1_name.split('_')[3]
#             ds2 = im1_name.split('_')[4]
#         elif 'S2' in im1_name and 'S2' in im2_name:
#             ds1 = im1_name[9:17]
#             ds2 = im2_name[9:17]
#         elif 'PS' in im1_name and 'PS' in im2_name:
#             ds1 = im1_name[3:11]
#             ds2 = im2_name[3:11]
#         else:
#             raise Exception('Optical data NOT supported yet!') 
#         print('Optical images coregistered.'); print()

#         ########### Load geogrid inputs and run (from runGeogridOptical) ################

#         # grab info from above
#         obj.startingX = trans[0]; obj.startingY = trans[3]
#         obj.XSize = trans[1]; obj.YSize = trans[5]
#         d0 = datetime.date(int(ds1[0:4]),int(ds1[4:6]),int(ds1[6:8]))
#         d1 = datetime.date(int(ds2[0:4]),int(ds2[4:6]),int(ds2[6:8]))
#         date_dt_base = d1 - d0
#         obj.repeatTime = date_dt_base.total_seconds()
#         obj.numberOfLines = ysize1; obj.numberOfSamples = xsize1
#         obj.gridSpacingX = dem_info['geoTransform'][1] # output grid spacing is the same as the DEM

#         # customize no data value and minimimum chip size
#         obj.nodata_out = NO_DATA_VAL
#         obj.chipSizeX0 = MINCHIPSIZE

#         # set raster paths and names
#         obj.dat1name = indir_m # first image
#         obj.demname = dem # DEM
#         obj.dhdxname = dhdx; obj.dhdyname = dhdy # surface slope
#         obj.vxname = vx; obj.vyname = vy # reference velocity
#         obj.srxname = srx; obj.sryname = sry # search range limits
#         obj.csminxname = csminx; obj.csminyname = csminy # min chip size
#         obj.csmaxxname = csmaxx; obj.csmaxyname = csmaxy # max chip size
#         obj.ssmname = ssm # stable surface mask
#         obj.winlocname = "window_location.tif"
#         obj.winoffname = "window_offset.tif"
#         obj.winsrname = "window_search_range.tif"
#         obj.wincsminname = "window_chip_size_min.tif"
#         obj.wincsmaxname = "window_chip_size_max.tif"
#         obj.winssmname = "window_stable_surface_mask.tif"
#         obj.winro2vxname = "window_rdr_off2vel_x_vec.tif"
#         obj.winro2vyname = "window_rdr_off2vel_y_vec.tif"

#         obj.runGeogrid() # RUN GEOGRID
#         print('Optical geogrid finished.'); print()

#     elif img_type == 'SAR': # SAR images
#         print('Processing SAR images with geogrid.'); print();
#         ############ Load SAR metadata from coreg_files ##################################
#         # Store sensing start info for 2nd SAR image (in temp_dir+secondary/)
#         frames = []
#         for swath in range(1,4):
#             inxml = os.path.join(temp_dir+'secondary/', 'IW{0}.xml'.format(swath))
#             if os.path.exists(inxml):
#                 pm = PM(); pm.configure(); ifg = pm.loadProduct(inxml) # load XML file
#                 frames.append(ifg)
#         info1_sensingStart = min([x.sensingStart for x in frames]) # store info1_sensingStart

#         # Load other info from 1st SAR image (in temp_dir+reference/)
#         del frames; frames = [] 
#         for swath in range(1,4):
#             inxml = os.path.join(temp_dir+'reference/', 'IW{0}.xml'.format(swath))
#             if os.path.exists(inxml):
#                 pm = PM(); pm.configure(); ifg = pm.loadProduct(inxml) # load XML file        
#                 frames.append(ifg)
#         print('SAR metadata loaded.'); print()

#         ############ Get merged orbit getMergedOrbit() ################################## 
#         # Create merged orbit
#         orb = Orbit(); orb.configure()
#         burst = frames[0].bursts[0]
#         # Add first burst orbit to begin with
#         for sv in burst.orbit:
#             orb.addStateVector(sv)
#         for pp in frames:
#             # Add all state vectors
#             for bb in pp.bursts:
#                 for sv in bb.orbit:
#                     if (sv.time< orb.minTime) or (sv.time > orb.maxTime):
#                         orb.addStateVector(sv)
#         print('Merged orbit created.'); print()

#         ############ Load geogrid inputs and run ###################################
#         obj = Geogrid()
#         obj.configure()

#         obj.orbit = orb # grab merged orbit
#         obj.startingRange = min([x.startingRange for x in frames])
#         obj.rangePixelSize = frames[0].bursts[0].rangePixelSize
#         obj.sensingStart = min([x.sensingStart for x in frames])
#         obj.prf = 1.0 / frames[0].bursts[0].azimuthTimeInterval
#         obj.lookSide = -1
#         obj.repeatTime = (info1_sensingStart - obj.sensingStart).total_seconds() # INFO1
#         obj.numberOfLines = int(np.round((max([x.sensingStop for x in frames])-obj.sensingStart).total_seconds()*obj.prf))+1
#         obj.numberOfSamples = int(np.round((max([x.farRange for x in frames])-obj.startingRange)/obj.rangePixelSize))+1
#         obj.gridSpacingX = dem_info['geoTransform'][1] # output grid spacing is the same as the DEM

#         # custom no data value and chip size
#         obj.nodata_out = NO_DATA_VAL
#         obj.chipSizeX0 = CHIPSIZE_M

#         # set raster paths and names
#         obj.demname = dem # DEM
#         obj.dhdxname = dhdx; obj.dhdyname = dhdy # surface slope
#         obj.vxname = vx; obj.vyname = vy # reference velocity
#         obj.srxname = srx; obj.sryname = sry # search range limmits
#         obj.csminxname = csminx; obj.csminyname = csminy # min chip size
#         obj.csmaxxname = csmaxx; obj.csmaxyname = csmaxy # max chip size
#         obj.ssmname = ssm # stable surface mask
#         obj.winlocname = "window_location.tif"
#         obj.winoffname = "window_offset.tif"
#         obj.winsrname = "window_search_range.tif"
#         obj.wincsminname = "window_chip_size_min.tif"
#         obj.wincsmaxname = "window_chip_size_max.tif"
#         obj.winssmname = "window_stable_surface_mask.tif"
#         obj.winro2vxname = "window_rdr_off2vel_x_vec.tif"
#         obj.winro2vyname = "window_rdr_off2vel_y_vec.tif"

#         obj.getIncidenceAngle() # SAR specific
#         obj.geogrid() # run geogrid
#         print('SAR geogrid finished.'); print();

#     else: # not OPT or SAR
#         print('Image type flag not recognized :', img_type)


#     ############ Move files produced to the out_path directory ##############
#     for file in os.listdir(os.getcwd()):
#         if file.startswith('window') and file.endswith('.tif'):
#             shutil.move(os.getcwd()+'/'+file, out_path+file)
#     print('Geogrid output files moved')

### For SAR images only: Coregister SAR images with topsApp.py

#### Put TG's custom topApp.xml on repo

Requires orbit and auxiliary files for the S1 images as well as the creation of 3 xml files (topsApp.xml, reference.xml, secondary.xml).

Prior to any processing, download S1 .SAFE files, auxiliary instrument files, and orbit files.

- Download .SAFE files from Alaska Satellite Facility: https://search.asf.alaska.edu/
- Download auxiliary instrument files from https://qc.sentinel1.copernicus.eu/aux_ins/
- Download orbit files from Copernicus Sentinels POD Data Hub: https://scihub.copernicus.eu/gnss/#/home

Although Sentinel-1 restituted orbits (RESORB) are of good quality, it is recommended to use the precise orbits (POEORB) when available. Typically, precise orbits are available with a 15 to 20-day lag from the day of the acquisition.

To proceed with automatic downloads from ASF, **wget** must be downloaded. Create a wget configuration file in the image directory using terminal following the [ASF API instructions](https://docs.asf.alaska.edu/api/tools/):

    echo 'http_user=CHANGE_ME' >> wget.conf
    echo 'http_password=CHANGE_ME' >> wget.conf
    chmod 600 wget.conf
    export WGETRC="wget.conf"
    
Also, create a netrc file containing the NASA Earthdata login info for topsApp.py.
    


In [16]:
# create .netrc file with Earthdata credentials
def create_netrc(netrc_name):
    from netrc import netrc
    from subprocess import Popen
    from getpass import getpass
    
    homeDir = os.path.expanduser("~")
    if os.path.exists(homeDir+'/'+netrc_name):
        print(netrc_name, ' with Earthdata credentials already exists.')

    urs = 'urs.earthdata.nasa.gov'    # Earthdata URL endpoint for authentication
    prompts = ['Enter NASA Earthdata Login Username: ',
               'Enter NASA Earthdata Login Password: ']
    # Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
    try:
        netrcDir = os.path.expanduser(f"~/{netrc_name}")
        netrc(netrcDir).authenticators(urs)[0]
    # Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
    except FileNotFoundError:
        Popen('touch {0}{2} | echo machine {1} >> {0}{2}'.format(homeDir + os.sep, urs, netrc_name), shell=True)
        Popen('echo login {} >> {}{}'.format(getpass(prompt=prompts[0]), homeDir + os.sep, netrc_name), shell=True)
        Popen('echo \'password {} \'>> {}{}'.format(getpass(prompt=prompts[1]), homeDir + os.sep, netrc_name), shell=True)
        # Set restrictive permissions
        Popen('chmod 0600 {0}{1}'.format(homeDir + os.sep, netrc_name), shell=True)

In [17]:
def download_orbits(SAFEzipfilepath, config_path, out_dir):
    # DOWNLOADS PRECISE ORBIT FILES FROM ASF
    # Requires installation of wget
    # INPUTS:
    #  SAFEzipfilepath: path to the SAFE file 
    #  configpath: 
    #  our_dir: path to the orbit directory where orbit files will be saved
    
    orb_type = 'aux_poeorb'
    zipname = SAFEzipfilepath.split('/')[-1] 
    time = zipname.split('_')[5]
    S1 = zipname.split('_')[0][-3:]
    scene_center_time = datetime.datetime.strptime(time,"%Y%m%dT%H%M%S")
    validity_start_time = scene_center_time-datetime.timedelta(days=1)
    validity_end_time =  scene_center_time+datetime.timedelta(days=1)
    
    # ASF URL
    url = "https://s1qc.asf.alaska.edu/%s/?validity_start=%s&validity_start=%s&validity_start=%s&sentinel1__mission=%s" % (orb_type, validity_start_time.strftime("%Y"),validity_start_time.strftime("%Y-%m"), validity_start_time.strftime("%Y-%m-%d"), S1)   
    content = ( urllib.request.urlopen(url).read()) # read results
    ii = re.findall('''href=["'](.[^"']+)["']''', content.decode('utf-8'))
    
    for i in ii :
        if '.EOF' in i:
            if (validity_start_time.strftime("%Y%m%d") in i) and (validity_end_time.strftime("%Y%m%d") in i) and (S1 in i):
                
                # if it doesn't already exist
                if not os.path.isfile(out_dir+i):
                    wget_cmd = 'export WGETRC="'+config_path+'"; '
                    wget_cmd += 'wget -c -P '+out_dir+' '
                    wget_cmd += "https://s1qc.asf.alaska.edu/aux_poeorb/"+i
    #                 print(wget_cmd)
                    subprocess.run(wget_cmd, shell=True,check=True)  
                    print(i+' downloaded.')
                else:
                    print(i+' already exists in orbit folder.')

In [18]:
# coregister pre-process the SAR images, generating XM: files
if img_type == 'SAR':
    create_netrc('.netrc') # create netrc file with Earthdata credentials if necessary
    
    # make a folder to hold intermediate files used for coregistration
    temp_dir = imgpath+'coreg_files/'
    if not os.path.exists(temp_dir): # if doesn't exist
        os.mkdir(temp_dir) # make it
        print('coreg_files folder created.'); print()
    else: # CLEAR ENTIRE FOLDER?
        for file in os.listdir(temp_dir): # if it already exists:
            if file.endswith('.xml'): # clear all the old XML files
                os.remove(temp_dir+file)
        print('coreg_files folder already exists. Old files cleared.'); print()
           
    # download the orbit files
    orbit_dir = imgpath+'orbits/'
    config_path = imgpath+'wget.conf' # path to the WGET config file
    if not os.path.isdir(orbit_dir): # if orbits folder doesn't exist
        os.mkdir(orbit_dir) # make it 
    if os.path.isfile(imgpath+'wget.conf'):
        print('Downloading orbit files: ')
        download_orbits(imgpath+im1_name, config_path, orbit_dir) # orbit file for 1st image
        download_orbits(imgpath+im2_name, config_path, orbit_dir) # orbit file for 2nd image
    else:
        print('Error: wget.conf not in ',imgpath)
    print()
    
    # move template XML files into the temporary folder
    for xmlname in ['topsApp.xml', 'reference.xml','secondary.xml']:
        if os.path.isfile(imgpath+xmlname):# if these xml template files exist
            shutil.copyfile(imgpath+xmlname, temp_dir+xmlname) # copy them into the temp folder
        else:
            print('Error: missing the template file '+xmlname); print()
            
    # auto update reference.xml & secondary.xml using XML Element Tree (ET) package
    tree1 = ET.parse(temp_dir+'reference.xml'); root1 = tree1.getroot() # first image
    tree2 = ET.parse(temp_dir+'secondary.xml'); root2 = tree2.getroot() # second image
    for prop in root1.iter(): # REFERENCE
        if prop.get('name') == 'orbit directory':
            prop.text = orbit_dir # set orbit directory to orbit_dir
        if prop.get('name') == 'safe':
            prop.text = imgpath+im1_name # set SAFE.zip file path
    tree1.write(temp_dir+'reference.xml') # over-write
    for prop in root2.iter(): # SECONDARY
        if prop.get('name') == 'orbit directory':
            prop.text = orbit_dir # set orbit directory to orbit_dir
        if prop.get('name') == 'safe':
            prop.text = imgpath+im2_name # set SAFE.zip file path
    tree2.write(temp_dir+'secondary.xml') # over-write
    
    # Run topsApp.py coregistration (will take some time to complete)
    runtopsapp = 'cd '+temp_dir+'; ' # change directory into temp folder
    runtopsapp += 'python3 '+imgpath+'topsApp.py --start=startup --end=rangecoreg' # tun topsapp.py
    subprocess.run(runtopsapp, shell=True, check=True)

.netrc  with Earthdata credentials already exists.
coreg_files folder already exists. Old files cleared.

S1A_OPER_AUX_POEORB_OPOD_20210411T121741_V20210321T225942_20210323T005942.EOF already exists in orbit folder.
S1A_OPER_AUX_POEORB_OPOD_20210423T121738_V20210402T225942_20210404T005942.EOF already exists in orbit folder.

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-08-26 14:01:37,385 - isce.insar - INFO - ISCE VERSION = 2.5.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210823, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.5.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20210823, CURRENT_SVN_REVISION = 
Step processing


CalledProcessError: Command 'cd /Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel1/coreg_files/; python3 /Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel1/topsApp.py --start=startup --end=rangecoreg' returned non-zero exit status 1.

### Set custom Geogrid parameters (Optical and SAR)

     input "dhdxname"/"dhdyname"                          -> output "winro2vxname"/"winro2vyname"
     input "dhdxname"/"dhdyname" and "vxname"/"vyname"    -> output "winro2vxname"/"winro2vyname" and "winoffname" 
     input "dhdxname"/"dhdyname" and "srxname"/"sryname"  -> output "winro2vxname"/"winro2vyname" and "winsrname"
     input "csminxname"/"csminyname"                      -> output "wincsminname"
     input "csmaxxname"/"csmaxyname"                      -> output "wincsmaxname"
     input "ssmname"                                      -> output "winssmname"

#### Best  MINCHIPSIZE >= SCALAR*PIXRES where SCALAR = 16 or some other power of 2

    LS = 200
    S2 = 160
    PS = 100

In [16]:
###### SET CUSTOM PARAMETERS FOR GEOGRID ################
dem = dempath+dem_outfile # path to the resampled DEM produced in the previous step (outfile name)
indir_m = imgpath+im1_name
indir_s = imgpath+im2_name
MINCHIPSIZE = 160 # smallest chip size allowed in image horizontal direction (in m)
NO_DATA_VAL = -32767 # no data value in the output products
temp_dir = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel1/coreg_files/'

if MINCHIPSIZE > CHIPSIZE_M:
    warning = 'Your minimum chip size for autoRIFT exceeds the output grid size (CHIPSIZE_M). '
    warning += 'Please increase the resampled DEM gridsize (CHIPSIZE_M).'
    print(warning)

# optional inputs (set as '' to leave blank)
dhdx = dempath+dhdx_outfile
dhdy = dempath+dhdy_outfile
vx = refvpath+vx_outfile
vy = refvpath+vy_outfile
# srx = refvpath+srx_outfile
# sry = refvpath+sry_outfile
csminx = refvpath+csminx_fname
csminy = refvpath+csminy_fname
csmaxx = refvpath+csmaxx_fname
csmaxy = refvpath+csmaxy_fname
ssm = refvpath+ssm_outfile # stable surface mask 

# dhdx = ''
# dhdy = ''
# vx = ''
# vy = ''
srx = ''
sry = ''
# csminx = ''
# csminy = ''
# csmaxx = ''
# csmaxy = ''
# ssm = ''
#########################################################

NameError: name 'dem_outfile' is not defined

In [23]:
run_geogrid_inhouse(out_path, img_type, indir_m, indir_s, MINCHIPSIZE, NO_DATA_VAL, dem, # required inputs
                    dhdx, dhdy, vx, vy, srx, sry, csminx, csminy, csmaxx, csmaxy, ssm, # optional inputs
                    temp_dir)

NameError: name 'indir_m' is not defined

In [ ]:
# ############ Clear all old geogrid files ##########################
# for file in os.listdir(out_path):
#     if file.startswith('window') and file.endswith('.tif'):
#         print('removed', file)
#         os.remove(out_path+file)
# print('Old files cleared.'); print()

# dem_info = gdal.Info(dem, format='json') # grab info from DEM
# print('Obtained DEM info.'); print()

# ############ Run geogrid optical or SAR ##########################
# if img_type == 'OPT': # Optical images
#     print('Processing optical images with geogrid.'); print()
#     obj = GeogridOptical() # initialize geogrid object
    
#     ############ Coregister the optical data (from coregisterLoadMetadataOptical) #############
#     x1a, y1a, xsize1, ysize1, x2a, y2a, xsize2, ysize2, trans = obj.coregister(indir_m, indir_s,0)
    
#     # grab dates from file names
#     if 'LC' in im1_name and 'LC' in im2_name:
#         ds1 = im1_name.split('_')[3]
#         ds2 = im1_name.split('_')[4]
#     elif 'S2' in im1_name and 'S2' in im2_name:
#         ds1 = im1_name[9:17]
#         ds2 = im2_name[9:17]
#     elif 'PS' in im1_name and 'PS' in im2_name:
#         ds1 = im1_name[3:11]
#         ds2 = im2_name[3:11]
#     else:
#         raise Exception('Optical data NOT supported yet!') 
#     print('Optical images coregistered.'); print()
    
#     ########### Load geogrid inputs and run (from runGeogridOptical) ################
    
#     # grab info from above
#     obj.startingX = trans[0]; obj.startingY = trans[3]
#     obj.XSize = trans[1]; obj.YSize = trans[5]
#     d0 = date(np.int(ds1[0:4]),np.int(ds1[4:6]),np.int(ds1[6:8]))
#     d1 = date(np.int(ds2[0:4]),np.int(ds2[4:6]),np.int(ds2[6:8]))
#     date_dt_base = d1 - d0
#     obj.repeatTime = date_dt_base.total_seconds()
#     obj.numberOfLines = ysize1; obj.numberOfSamples = xsize1
#     obj.gridSpacingX = dem_info['geoTransform'][1] # output grid spacing is the same as the DEM
    
#     # customize no data value and minimimum chip size
#     obj.nodata_out = NO_DATA_VAL
#     obj.chipSizeX0 = CHIPSIZE_M
    
#     # set raster paths and names
#     obj.dat1name = indir_m # first image
#     obj.demname = dem # DEM
#     obj.dhdxname = dhdx; obj.dhdyname = dhdy # surface slope
#     obj.vxname = vx; obj.vyname = vy # reference velocity
#     obj.srxname = srx; obj.sryname = sry # search range limits
#     obj.csminxname = csminx; obj.csminyname = csminy # min chip size
#     obj.csmaxxname = csmaxx; obj.csmaxyname = csmaxy # max chip size
#     obj.ssmname = ssm # stable surface mask
#     obj.winlocname = "window_location.tif"
#     obj.winoffname = "window_offset.tif"
#     obj.winsrname = "window_search_range.tif"
#     obj.wincsminname = "window_chip_size_min.tif"
#     obj.wincsmaxname = "window_chip_size_max.tif"
#     obj.winssmname = "window_stable_surface_mask.tif"
#     obj.winro2vxname = "window_rdr_off2vel_x_vec.tif"
#     obj.winro2vyname = "window_rdr_off2vel_y_vec.tif"
    
#     obj.runGeogrid() # RUN GEOGRID
#     print('Optical geogrid finished.'); print()
    
# elif img_type == 'SAR': # SAR images
#     print('Processing SAR images with geogrid.'); print();
#     ############ Load SAR metadata from coreg_files ##################################
#     # Store sensing start info for 2nd SAR image (in temp_dir+secondary/)
#     frames = []
#     for swath in range(1,4):
#         inxml = os.path.join(temp_dir+'secondary/', 'IW{0}.xml'.format(swath))
#         if os.path.exists(inxml):
#             pm = PM(); pm.configure(); ifg = pm.loadProduct(inxml) # load XML file
#             frames.append(ifg)
#     info1_sensingStart = min([x.sensingStart for x in frames]) # store info1_sensingStart
    
#     # Load other info from 1st SAR image (in temp_dir+reference/)
#     del frames; frames = [] 
#     for swath in range(1,4):
#         inxml = os.path.join(temp_dir+'reference/', 'IW{0}.xml'.format(swath))
#         if os.path.exists(inxml):
#             pm = PM(); pm.configure(); ifg = pm.loadProduct(inxml) # load XML file        
#             frames.append(ifg)
#     print('SAR metadata loaded.'); print()
    
#     ############ Get merged orbit getMergedOrbit() ################################## 
#     # Create merged orbit
#     orb = Orbit(); orb.configure()
#     burst = frames[0].bursts[0]
#     # Add first burst orbit to begin with
#     for sv in burst.orbit:
#         orb.addStateVector(sv)
#     for pp in frames:
#         # Add all state vectors
#         for bb in pp.bursts:
#             for sv in bb.orbit:
#                 if (sv.time< orb.minTime) or (sv.time > orb.maxTime):
#                     orb.addStateVector(sv)
#     print('Merged orbit created.'); print()
                    
#     ############ Load geogrid inputs and run ###################################
#     obj = Geogrid()
#     obj.configure()
    
#     obj.orbit = orb # grab merged orbit
#     obj.startingRange = min([x.startingRange for x in frames])
#     obj.rangePixelSize = frames[0].bursts[0].rangePixelSize
#     obj.sensingStart = min([x.sensingStart for x in frames])
#     obj.prf = 1.0 / frames[0].bursts[0].azimuthTimeInterval
#     obj.lookSide = -1
#     obj.repeatTime = (info1_sensingStart - obj.sensingStart).total_seconds() # INFO1
#     obj.numberOfLines = int(np.round((max([x.sensingStop for x in frames])-obj.sensingStart).total_seconds()*obj.prf))+1
#     obj.numberOfSamples = int(np.round((max([x.farRange for x in frames])-obj.startingRange)/obj.rangePixelSize))+1
#     obj.gridSpacingX = dem_info['geoTransform'][1] # output grid spacing is the same as the DEM
    
#     # custom no data value and chip size
#     obj.nodata_out = NO_DATA_VAL
#     obj.chipSizeX0 = CHIPSIZE_M
    
#     # set raster paths and names
#     obj.demname = dem # DEM
#     obj.dhdxname = dhdx; obj.dhdyname = dhdy # surface slope
#     obj.vxname = vx; obj.vyname = vy # reference velocity
#     obj.srxname = srx; obj.sryname = sry # search range limmits
#     obj.csminxname = csminx; obj.csminyname = csminy # min chip size
#     obj.csmaxxname = csmaxx; obj.csmaxyname = csmaxy # max chip size
#     obj.ssmname = ssm # stable surface mask
#     obj.winlocname = "window_location.tif"
#     obj.winoffname = "window_offset.tif"
#     obj.winsrname = "window_search_range.tif"
#     obj.wincsminname = "window_chip_size_min.tif"
#     obj.wincsmaxname = "window_chip_size_max.tif"
#     obj.winssmname = "window_stable_surface_mask.tif"
#     obj.winro2vxname = "window_rdr_off2vel_x_vec.tif"
#     obj.winro2vyname = "window_rdr_off2vel_y_vec.tif"
    
#     obj.getIncidenceAngle() # SAR specific
#     obj.geogrid() # run geogrid
#     print('SAR geogrid finished.'); print();
    
# else: # not OPT or SAR
#     print('Image type flag not recognized :', img_type)
    

# ############ Move files produced to the out_path directory ##############
# for file in os.listdir(os.getcwd()):
#     if file.startswith('window') and file.endswith('.tif'):
#         shutil.move(os.getcwd()+'/'+file, out_path+file)
# print('Geogrid output files moved')

## 3) Run autoRIFT with new parameters

In [12]:
# identify files produced from geogrid
gp = out_path

# remove all empty grids
for grid in os.listdir(gp): 
    if grid.startswith('window') and grid.endswith('.tif'):
        reader = rio.open(gp+grid) # read dataset
        data_found = False 
        for band in range(1,reader.count+1):
            testband = reader.read(band) # read in the band
            if np.count_nonzero(testband[testband != NO_DATA_VAL]) > 0:
                data_found = True
        if not data_found:
            print(grid, 'has no data. Removed.')
            os.remove(gp+grid)
        else:
            print(grid)

In [ ]:
# fill in AutoRIFT parameters using the files - make all of these function arguments
mpflag = 0 # leave multiprocessing off

# GRID LOCATION (required) from window_location.tif
grid_location = rio.open(gp+'window_location.tif')
xGrid = grid_location.read(1) # 1st band in window location
yGrid = grid_location.read(2) # 2nd band in window location

# optional parameters (default None or zero until filled)
init_offset = None; search_range = None
chip_size_min = None; chip_size_max = None
offset2vx = None; offset2vy = None; stable_surface_mask = None
Dx0 = None; Dy0 = None; CSMINx0 = None
SRx0 = None; SRy0 = None;
CSMAXx0 = None; CSMAXy0 = None; SSM = None
noDataMask = np.zeros(xGrid.shape).astype(int)

if os.path.exists(gp+'window_offset.tif'): # Dx0 and Dy0 from window_offset.tif
    init_offset = rio.open(gp+'window_offset.tif')
    Dx0 = init_offset.read(1); Dy0 = init_offset.read(2)
if os.path.exists(gp+'window_search_range.tif'): # SRx0 and SRy0 from window_search_range.tif
    search_range = rio.open(gp+'window_search_range.tif')
    SRx0 = search_range.read(1); SRy0 = search_range.read(2)
if os.path.exists(gp+'window_chip_size_min.tif'): # CSMINx0 and CSMINy0 from window_chip_size_min.tif
    chip_size_min = rio.open(gp+'window_chip_size_min.tif')
    CSMINx0 = chip_size_min.read(1); CSMINy0 = chip_size_min.read(2)
if os.path.exists(gp+'window_chip_size_max.tif'): # CSMAXx0 and CSMAXy0 from window_chip_size_max.tif
    chip_size_max = rio.open(gp+'window_chip_size_max.tif')
    CSMAXx0 = chip_size_max.read(1); CSMAXy0 = chip_size_max.read(2)
if os.path.exists(gp+'window_rdr_off2vel_x_vec.tif'): # offset2vx from window_rdr_off2vel_x_vec.tif
    offset2vx = gp+'window_rdr_off2vel_x_vec.tif' # path to be read in with GDAL
if os.path.exists(gp+'window_rdr_off2vel_y_vec.tif'): # offset2vy from window_rdr_off2vel_y_vec.tif
    offset2vy = gp+'window_rdr_off2vel_y_vec.tif' 
if os.path.exists(gp+'window_stable_surface_mask.tif'): # noDataMask from window_stable_surface_mask.tif
    stable_surface_mask = rio.open(gp+'window_stable_surface_mask.tif')
    noDataMask = stable_surface_mask.read(1)
    
# other parameters
nodata = NO_DATA_VAL # use same as in previous steps
geogrid_run_info=None
print('AutoRIFT parameters loaded.')



In [ ]:
######################### CHOOSE FILTERS & SAMPLING ########################
# Filters:
# options are HPS (high pass), WAL (wallis), SOB (sobel), DB (logarithmic operator)
FILTER = 'WAL'
WALLISFILTERWIDTH = 32 # only for wallis filter, must be a power of 2

# Sampling:
SPARSE_SEARCH_SAMPLE_RATE = 16 # how many samples to skip to speed up processing
OVERSAMPLE_RATIO = 0 # enter in a constant scalar or 0 for default parameters
############################################################################
print('Filters and autoRIFT sample rates chosen.')

In [ ]:
# run autoRIFT with function
run_autoRIFT_inhouse(out_path, img_type, mpflag, xGrid, yGrid, # required parameters
                         FILTER, WALLISFILTERWIDTH, SPARSE_SEARCH_SAMPLE_RATIO, OVERSAMPLE_RATIO,
                         Dx0, Dy0, CSMINx0, SRx0, SRy0, CSMAXx0, CSMAXy0, SSM, # optional parameters
                         noDataMask, nodataval, geogrid_run_info)

In [ ]:
# AutoRIFT  
def run_autoRIFT_inhouse(out_path, img_type, mpflag, xGrid, yGrid, # required parameters
                         FILTER, WALLISFILTERWIDTH, SPARSE_SEARCH_SAMPLE_RATIO, OVERSAMPLE_RATIO, MINCHIPSIZE,
                         Dx0, Dy0, CSMINx0, SRx0, SRy0, CSMAXx0, CSMAXy0, SSM, # optional parameters
                         noDataMask, nodataval, geogrid_run_info):
    CHIPSIZE_M = MINCHIPSIZE # set minimum chip size equal
    
    # requires grid location from geogrid
    origSize = xGrid.shape # grab original size from xGrid
    
    if img_type == 'OPT': ############# OPTICAL SETTINGS ############################# 
        print('Processing optical images with autoRIFT.'); print()
        optflag = 1 # turn on optical flag
        # Coregister and read in the two images (from loadProductOptical())
        obj = GeogridOptical()
        x1a, y1a, xsize1, ysize1, x2a, y2a, xsize2, ysize2, trans = obj.coregister(indir_m, indir_s,0)

        # read dates from filenames
        if 'LC' in indir_m and 'LC' in indir_s:
            ds1 = indir_m.split('/')[-1].split('_')[3]; ds2 = indir_s.split('/')[-1].split('_')[3]
            sat = 'LS'
        elif 'S2' in indir_m and 'S2' in indir_s:
            ds1 = indir_m.split('/')[-1][9:17]; ds2 = indir_s.split('/')[-1][9:17]
            sat = 'S2'
        elif 'PS' in im1_name and 'PS' in im2_name:
            ds1 = indir_m.split('/')[-1].split('_')[1]
            ds2 = indir_s.split('/')[-1].split('_')[1]
            sat = 'PS'
        else:
            raise Exception('Optical data NOT supported yet!')

        # read in the images
        DS1 = gdal.Open(indir_m); DS2 = gdal.Open(indir_s)
        I1 = DS1.ReadAsArray(xoff=x1a, yoff=y1a, xsize=xsize1, ysize=ysize1)
        I1 = I1.astype(np.float32)
        I2 = DS2.ReadAsArray(xoff=x2a, yoff=y2a, xsize=xsize2, ysize=ysize2)
        I2 = I2.astype(np.float32)
        DS1=None; DS2=None # clear DS1 and DS2

        # Initialize autoRIFT object (from runAutorift())
        obj = autoRIFT_ISCE()
        obj.configure()

    elif img_type == 'SAR': ############# SAR SETTINGS #############################  
        print('Processing SAR images with autoRIFT.'); print()
        optflag = 0 # turn off opt flag
        # Read in the two SAR images (from loadProduct())
        img1 = IML.mmapFromISCE(filename1, logging); I1 = IMG.bands[0]
        img2 = IML.mmapFromISCE(filename2, logging); I2 = IMG.bands[0]
        I1 = np.abs(I1); I2 = np.abs(I2) # SAR amplitude only
        
    else:
        print("Image type not recognized. Use either 'OPT' or 'SAR'.")
        
    ############# Initialize autoRIFT object (from runAutorift()) ##################
    obj = autoRIFT_ISCE()
    obj.configure()
    
    obj.MultiThread = mpflag # multiprocessing
    obj.I1 = I1; obj.I2 = I2 # assign the images
    obj.xGrid = xGrid; obj.yGrid = yGrid # assign the grid 

    # GENERATE NO DATA MASK
    # where offset searching will be skipped based on 
    # 1) imported nodata mask and/or 2) zero values in the image
    for ii in range(obj.xGrid.shape[0]):
        for jj in range(obj.xGrid.shape[1]):
            if (obj.yGrid[ii,jj] != nodata)&(obj.xGrid[ii,jj] != nodata):
                if (I1[obj.yGrid[ii,jj]-1,obj.xGrid[ii,jj]-1]==0)|(I2[obj.yGrid[ii,jj]-1,obj.xGrid[ii,jj]-1]==0):
                    noDataMask[ii,jj] = True
                    
    # SEARCH RANGE
    if SRx0 is None:
        # default is a zero array
#        ###########     uncomment to customize SearchLimit based on velocity distribution 
        if Dx0 is not None:
            obj.SearchLimitX = np.int32(4+(25-4)/(np.max(np.abs(Dx0[np.logical_not(noDataMask)]))-np.min(np.abs(Dx0[np.logical_not(noDataMask)])))*(np.abs(Dx0)-np.min(np.abs(Dx0[np.logical_not(noDataMask)]))))
        else:
            obj.SearchLimitX = 15
        obj.SearchLimitY = 15
#        ###########
        obj.SearchLimitX = obj.SearchLimitX * np.logical_not(noDataMask)
        obj.SearchLimitY = obj.SearchLimitY * np.logical_not(noDataMask)
    else:
        obj.SearchLimitX = SRx0
        obj.SearchLimitY = SRy0
       ############ add buffer to search range
        obj.SearchLimitX[obj.SearchLimitX!=0] = obj.SearchLimitX[obj.SearchLimitX!=0] + 2
        obj.SearchLimitY[obj.SearchLimitY!=0] = obj.SearchLimitY[obj.SearchLimitY!=0] + 2
    
    # CHIP SIZE
    if CSMINx0 is not None:
        obj.ChipSizeMaxX = CSMAXx0
        obj.ChipSizeMinX = CSMINx0
        
        gridspacingx = MINCHIPSIZE # use the grid spacing from above
        chipsizex0 = MINCHIPSIZE
        pixsizex = trans[1] # grab from coregister function
    
        obj.ChipSize0X = int(np.ceil(chipsizex0/pixsizex/4)*4)
        obj.GridSpacingX = int(obj.ChipSize0X*gridspacingx/chipsizex0)

        RATIO_Y2X = CSMINy0/CSMINx0
        obj.ScaleChipSizeY = np.median(RATIO_Y2X[(CSMINx0!=nodata)&(CSMINy0!=nodata)])
#         obj.ScaleChipSizeY = 1 # USE SCALE OF 1 for square pixels
    else:
        if ((optflag == 1)&(xGrid is not None)):
            obj.ChipSizeMaxX = 32 # pixels
            obj.ChipSizeMinX = 16 # pixels
            obj.ChipSize0X = 16 # pixels
    
    # DOWNSTREAM SEARCH OFFSET
    if Dx0 is not None:
        obj.Dx0 = Dx0
        obj.Dy0 = Dy0
    else:
        obj.Dx0 = obj.Dx0 * np.logical_not(noDataMask)
        obj.Dy0 = obj.Dy0 * np.logical_not(noDataMask)

    # REPLACE NO DATA VALUES WITH 0
    obj.xGrid[noDataMask] = 0
    obj.yGrid[noDataMask] = 0
    obj.Dx0[noDataMask] = 0
    obj.Dy0[noDataMask] = 0
    if SRx0 is not None:
        obj.SearchLimitX[noDataMask] = 0
        obj.SearchLimitY[noDataMask] = 0
    if CSMINx0 is not None:
        obj.ChipSizeMaxX[noDataMask] = 0
        obj.ChipSizeMinX[noDataMask] = 0
    
    # convert azimuth offset to vertical offset as used in autoRIFT convention for SAR images
    if optflag == 0:
        obj.Dy0 = -1 * obj.Dy0
        
    ############## AutoRIFT Pre-processing (from runAutorift()) ############################
    t1 = time.time()
    print("Pre-process Start!!!")
    
    # FILTERING:
    if FILTER == 'WAL': 
        obj.preprocess_filt_wal() # WALLIS FILTER
#         obj.zeroMask = 1 # removes edges
        obj.WallisFilterWidth = WALLISFILTERWIDTH # optional, default supposedly 21
    elif FILTER == 'HPS':
        obj.preprocess_filt_hps() # HIGH PASS FILTER
    elif FILTER == 'SOB':
        obj.preprocess_filt_sob() # SOBEL FILTER
    elif FILTER == 'LAP':
        obj.preprocess_filt_lap()
    elif FILTER == 'DB':
        obj.preprocess_db() # LOGARITHMIC OPERATOR (NO FILTER), FOR TOPOGRAPHY
    else:
        print(FILTER, 'not recognized. Using default high pass filter instead.')
        obj.preprocess_filt_hps() # HIGH PASS FILTER
        
    print("Pre-process Done!!!")
    print(time.time()-t1)
    
    # CONVERT TO UNIFORM DATA TYPE
    t1 = time.time()
#    obj.DataType = 0
    obj.uniform_data_type()
    print("Uniform Data Type Done!!!")
    print(time.time()-t1)
    
    # OTHER :
    obj.sparseSearchSampleRate = 1
#    obj.colfiltChunkSize = 4

    obj.OverSampleRatio = 64
    if CSMINx0 is not None:
        obj.OverSampleRatio = {obj.ChipSize0X:16,obj.ChipSize0X*2:32,obj.ChipSize0X*4:64,obj.ChipSize0X*8:64}
    
    # SEE ORIGINAL CODE TO EXPORT PREPROCESSED IMAGES
    
    ####################### Run AutoRIFT (from runAutorift())  ############################
    t1 = time.time()
    print("AutoRIFT Start!!!")
    obj.runAutorift()
    print("AutoRIFT Done!!!")
    print(time.time()-t1)
    
    kernel = np.ones((3,3),np.uint8)
    noDataMask = cv2.dilate(noDataMask.astype(np.uint8),kernel,iterations = 1)
    noDataMask = noDataMask.astype(np.bool)

    # AT THIS POINT, THESE VARIABLES WILL BE CREATED:
    # obj.Dx, obj.Dy, obj.InterpMask, obj.ChipSizeX, obj.GridSpacingX, 
    # obj.ScaleChipSizeY, obj.SearchLimitX, obj.SearchLimitY, obj.origSize, noDataMask
    
    # PLOT RESULTS
    fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(15,5))
    im1 = ax1.imshow(obj.Dx); ax1.set_title('Dx'); fig.colorbar(im1, ax=ax1)
    im2 = ax2.imshow(obj.Dy); ax2.set_title('Dy'); fig.colorbar(im2, ax=ax2)
    im3 = ax3.imshow(np.sqrt((obj.Dx**2) + (obj.Dy**2))); ax3.set_title('D_total'); fig.colorbar(im3,ax=ax3)
    plt.suptitle(ds1+' to '+ds2)
    plt.show()

    ####################### Write outputs (from runAutorift())  ############################
    t1 = time.time()
    print("Write Outputs Start!!!")
          
    # Write text file with parameters
    f =  open(out_path+'parameters_'+ds1+'_'+ds2+'_'+str(CHIPSIZE_M)+'m_'+sat+'.txt', 'w')
    f.write('Geogrid/AutoRIFT parameters for offset_'+ds1+'_'+ds2+'_'+str(CHIPSIZE_M)+'m_'+sat+'.tif:')
    f.write('NO_DATA_VAL: '+str(NO_DATA_VAL))
    f.write('Min chip size: '+str(MINCHIPSIZE))
    f.write('DEM: '+dem)
    f.write('dhdx: '+dhdx); f.write('dhdy: '+dhdy)
    f.write('vx: '+vx); f.write('vy: '+vy)
    f.write('srx: '+srx); f.write('sry: '+sry)
    f.write('csminx: '+csminx); f.write('csminy: '+csminy)
    f.write('csmaxx: '+csmaxx); f.write('csmaxy: '+csmaxy)
    f.write('stable surface mask: '+ssm)
    f.write('FILTER: '+FILTER)
    f.write('WALLISFILTERWIDTH: '+str(WALLISFILTERWIDTH))
    f.write('Spare search sample rate: '+str(SPARSE_SEARCH_SAMPLE_RATE))
    f.write('Oversample ratio: '+str(OVERSAMPLE_RATIO))
    if offset2vx is not None and offset2vy is not None:
        f.write('Velocity.TIF file created.')
    else:
        f.write('Velocity.TIF not created.')
    f.close() # close the parameter text file
          
    # open the window_location.tif file to gdalinfo
    ds = gdal.Open(gp+'window_location.tif')
    tran = ds.GetGeoTransform()
    proj = ds.GetProjection()
    srs = ds.GetSpatialRef()
    
    # initialize arrays
    DX = np.zeros(origSize,dtype=np.float32) * np.nan; DY = np.zeros(origSize,dtype=np.float32) * np.nan
    INTERPMASK = np.zeros(origSize,dtype=np.float32); CHIPSIZEX = np.zeros(origSize,dtype=np.float32)
    SEARCHLIMITX = np.zeros(origSize,dtype=np.float32); SEARCHLIMITY = np.zeros(origSize,dtype=np.float32)
    
    # fill in arays
    Dx = obj.Dx; Dy = obj.Dy; InterpMask = obj.InterpMask; ChipSizeX = obj.ChipSizeX
    SearchLimitX = obj.SearchLimitX; SearchLimitY = obj.SearchLimitY
    DX[0:Dx.shape[0],0:Dx.shape[1]] = Dx;  DY[0:Dy.shape[0],0:Dy.shape[1]] = Dy
    INTERPMASK[0:InterpMask.shape[0],0:InterpMask.shape[1]] = InterpMask
    CHIPSIZEX[0:ChipSizeX.shape[0],0:ChipSizeX.shape[1]] = ChipSizeX
    SEARCHLIMITX[0:SearchLimitX.shape[0],0:SearchLimitX.shape[1]] = SearchLimitX
    SEARCHLIMITY[0:SearchLimitY.shape[0],0:SearchLimitY.shape[1]] = SearchLimitY
    
    # mask out no data
    DX[noDataMask] = np.nan; DY[noDataMask] = np.nan
    INTERPMASK[noDataMask] = 0; CHIPSIZEX[noDataMask] = 0
    SEARCHLIMITX[noDataMask] = 0; SEARCHLIMITY[noDataMask] = 0
    if SSM is not None:
        SSM[noDataMask] = False
    DX[SEARCHLIMITX == 0] = np.nan; DY[SEARCHLIMITX == 0] = np.nan
    INTERPMASK[SEARCHLIMITX == 0] = 0; CHIPSIZEX[SEARCHLIMITX == 0] = 0
    if SSM is not None:
        SSM[SEARCHLIMITX == 0] = False

    # SAVE TO OFFSET.MAT FILE
    sio.savemat('offset_'+ds1+'_'+ds2+'_'+str(CHIPSIZE_M)+'m_'+sat+'.mat', # offset mat filename
                {'Dx':DX,'Dy':DY,'InterpMask':INTERPMASK,'ChipSizeX':CHIPSIZEX})
    print('Offset.mat written.')
    
    # CREATE THE GEOTIFFS
    driver = gdal.GetDriverByName('GTiff')
    
    # OFFSET.TIF
    outRaster = driver.Create("offset_"+ds1+'_'+ds2+'_'+str(CHIPSIZE_M)+'m_'+sat+".tif", # offset filename
                              int(xGrid.shape[1]), int(xGrid.shape[0]), 5, gdal.GDT_Float32)
    outRaster.SetGeoTransform(tran); outRaster.SetProjection(proj) # projections
    outband = outRaster.GetRasterBand(1); outband.WriteArray(DX) # DX
    outband.FlushCache()
    outband = outRaster.GetRasterBand(2); outband.WriteArray(DY) # DY
    outband.FlushCache()
    outband = outRaster.GetRasterBand(3); outband.WriteArray(np.sqrt((DX**2) + (DY**2))) # DY
    outband.FlushCache()
    outband = outRaster.GetRasterBand(4); outband.WriteArray(INTERPMASK) # INTERPMASK
    outband.FlushCache()
    outband = outRaster.GetRasterBand(5); outband.WriteArray(CHIPSIZEX) # CHIPSIZE
    outband.FlushCache()
    del outRaster
    print('Offset.tif written.')
    
    # VELOCITY.TIF
    if offset2vx is not None and offset2vy is not None:
        ds = gdal.Open(offset2vx) #### VX
        band = ds.GetRasterBand(1); offset2vx_1 = band.ReadAsArray()
        band = ds.GetRasterBand(2); offset2vx_2 = band.ReadAsArray()
        if ds.RasterCount > 2:
                band = ds.GetRasterBand(3)
                offset2vr = band.ReadAsArray()
        else:
                offset2vr = None
        band=None; ds=None
        offset2vx_1[offset2vx_1 == nodata] = np.nan
        offset2vx_2[offset2vx_2 == nodata] = np.nan

        ds = gdal.Open(offset2vy) #### VY
        band = ds.GetRasterBand(1); offset2vy_1 = band.ReadAsArray()
        band = ds.GetRasterBand(2); offset2vy_2 = band.ReadAsArray()
        if ds.RasterCount > 2:
                band = ds.GetRasterBand(3)
                offset2va = band.ReadAsArray()
        else:
                offset2va = None
        band=None; ds=None
        offset2vy_1[offset2vy_1 == nodata] = np.nan; offset2vy_2[offset2vy_2 == nodata] = np.nan
        
        if offset2va is not None:
            offset2va[offset2va == nodata] = np.nan

        VX = offset2vx_1 * DX + offset2vx_2 * DY
        VY = offset2vy_1 * DX + offset2vy_2 * DY
        VX = VX.astype(np.float32); VY = VY.astype(np.float32)

        outRaster = driver.Create("velocity_"+ds1+'_'+ds2+'_'+str(CHIPSIZE_M)+'m_'+sat+".tif", # velocity filename
                                  int(xGrid.shape[1]), int(xGrid.shape[0]), 3, gdal.GDT_Float32)
        outRaster.SetGeoTransform(tran); outRaster.SetProjection(proj)
        outband = outRaster.GetRasterBand(1); outband.WriteArray(VX) # VX
        outband.FlushCache()
        outband = outRaster.GetRasterBand(2); outband.WriteArray(VY) # VY
        outband.FlushCache()
        outband = outRaster.GetRasterBand(3); outband.WriteArray(np.sqrt((VX**2) + (VY**2))) # V
        outband.FlushCache()
        del outRaster
        print('Velocity.tif written.')
    
    print("Write Outputs Done!!!")
    print(time.time()-t1)
    
    # Move files produced to the out_path directory
    for file in os.listdir(os.getcwd()):
        if 'offset' in file or ('velocity' in file and '.tif' in file):
            shutil.move(os.getcwd()+'/'+file, out_path+file)

In [ ]:
# img, dataName, metaName = loadImage(temp_dir+'reference/IW1.xml')
# dataName = dataName+'.'

In [ ]:
# img.width

In [ ]:
# IML.memmap(dataName, nchannels=img.bands,
#             nxx=img.width, nyy=img.length, scheme=img.scheme,
#             dataType=IML.NUMPY_type(img.dataType))

In [ ]:
# loadProduct(temp_dir+'coarse_coreg/overlaps/IW1/burst_bot_02_03.slc.vrt')

In [ ]:
# memmap(dataName+'.vrt', nchannels=img.bands, nxx=img.width, nyy=img.length, scheme=img.scheme,
#             dataType=IML.NUMPY_type(img.dataType))

In [ ]:
# fname = temp_dir+'coarse_coreg/overlaps/IW1/burst_bot_02_03.slc.vrt'

In [ ]:
# os.listdir(temp_dir+'reference/IW1/')

In [ ]:
# mmapFromISCE(temp_dir+'reference/IW1/burst_01.slc.vrt')

# LOAD THE REFERENCE AND SECONDARY IMAGES (ISCE OR GDAL), REQUIRES .XML AND .VRT (only burst xmls?)

In [40]:
dataName

'/Users/jukesliu/Documents/PhD/ISCE_Aug2022/insar/secondary/IW1'

In [52]:
img, dataName, metaName = loadImage('/Users/jukesliu/Documents/PhD/ISCE_Aug2022/dense_offsets_ampcor/SK_test/fine_interferogram/IW2.xml')
img.width




In [39]:
import logging
from imageMath import IML
    
def loadProduct(filename):
    '''
    Load the product using Product Manager.
    '''
    import isce
    import logging
    from imageMath import IML

    IMG = IML.mmapFromISCE(filename, logging)
    img = IMG.bands[0]
#    pdb.set_trace()
    return img

def mmapFromISCE(fname, logger=None):
    '''
    Create a file mmap object using information in an ISCE XML.
    '''
    try:
        img, dataName, metaName = loadImage(fname)
        isceFile = True
        dataName = dataName+'.vrt'
    except:
        try:
            img = loadGDALImage(fname)
            isceFile=False
            dataName = fname
        except:
            raise Exception('Input file: {0} should either be an ISCE image / GDAL image. Appears to be neither'.format(fname))

#     if logger is not None:
#         logger.debug('Creating readonly ISCE mmap with \n' +
#             'file = %s \n'%(dataName) + 
#             'bands = %d \n'%(img.bands) + 
#             'width = %d \n'%(img.width) + 
#             'length = %d \n'%(img.length)+
#             'scheme = %s \n'%(img.scheme) +
#             'dtype = %s \n'%(img.dataType))
    
    if isceFile:
        mObj = IML.memmap(dataName, nchannels=img.bands,
            nxx=img.width, nyy=img.length, scheme=img.scheme,
            dataType=IML.NUMPY_type(img.dataType))
    else:
        mObj = memmapGDAL(dataName)

    return mObj

def loadImage(fname):
    '''
    Load into appropriate image object.
    '''
    try:
        import iscesys
        import isceobj
        from iscesys.Parsers.FileParserFactory import createFileParser
    except:
        raise ImportError('ISCE has not been installed or is not importable')

    if not fname.endswith('.xml'):
        dataName = fname
        metaName = fname[:-4] + '.xml'
    else:
        metaName = fname
        dataName = os.path.splitext(fname)[0]

    parser = createFileParser('xml')
    prop,fac,misc = parser.parse(metaName)

    if 'reference' in prop:
        img=isceobj.createDemImage()
        img.init(prop,fac,misc)
    elif 'number_good_bytes' in prop:
        img = isceobj.createRawImage()
        img.init(prop,fac,misc)
    else:
        img = isceobj.createImage()
        img.init(prop,fac,misc)

    img.setAccessMode('READ')
    return img, dataName, metaName

def loadGDALImage(fname):
    '''
    Similar to loadImage but only returns metadata.
    '''

    from osgeo import gdal

    class Dummy(object):
        pass


    ds = gdal.Open(fname, gdal.GA_ReadOnly)
    drv = ds.GetDriver()
    bnd = ds.GetRasterBand(1)

    img = Dummy()
    img.bands = ds.RasterCount 
    img.width = ds.RasterXSize
    img.length = ds.RasterYSize
    img.scheme = drv.GetDescription()
    if bnd.DataType == 10:
        img.dataType = 'CFLOAT'
    else:
        img.dataType = gdal.GetDataTypeByName(bnd.DataType)

    bnd = None
    drv = None
    ds = None

    return img


In [ ]:
# class memmapGDAL(object):
#     '''
#     Create a memmap like object from GDAL.
#     '''

#     from osgeo import gdal

#     class BandWrapper:
#         '''
#         Wrap a GDAL band in a numpy like slicable object.
#         '''

#         def __init__(self, dataset, band):
#             '''
#             Init from a GDAL raster band.
#             '''

#             self.data = dataset.GetRasterBand(band)
#             self.width = dataset.RasterXSize
#             self.length = data.RasterYSize

#         def __getitem__(self, *args):
            
#             xmin = max(int(args[0][1].start),0)
#             xmax = min(int(args[0][1].stop)+xmin, self.width) - xmin
#             ymin = max(int(args[0][0].start),0)
#             ymax = min(int(args[0][1].stop)+ymin, self.length) - ymin

#             res = self.data.ReadAsArray(xmin, ymin, xmax,ymax)
#             return res

#         def __del__(self):
#             self.data = None


#     def __init__(self, fname):
#         '''
#         Constructor.
#         '''
        
#         self.name = fname
#         self.data = gdal.Open(self.name, gdal.GA_ReadOnly)
#         self.width = self.data.RasterXSize
#         self.length = self.data.RasterYSize

#         self.bands = []
#         for ii in range(self.data.RasterCount):
#             self.bands.append( BandWrapper(self.data, ii+1))

#     def __del__(self):
#         self.data = None

# class memmap(object):
#     '''Create the memap object.'''
#     def __init__(self,fname, mode='readonly', nchannels=1, nxx=None, nyy=None, scheme='BSQ', dataType='f'):
#         '''Init function.'''

#         fsize = np.zeros(1, dtype=dataType).itemsize

#         if nxx is None:
#             raise ValueError('Undefined file width for : %s'%(fname))

#         if mode=='write':
#             if nyy is None:
#                 raise ValueError('Undefined file length for opening file: %s in write mode.'%(fname))
#         else:
#             try:
#                 nbytes = os.path.getsize(fname)
#             except:
#                 raise ValueError('Non-existent file : %s'%(fname))

# #            if nyy is None:
# #                nyy = nbytes//(fsize*nchannels*nxx)
# #
# #                if (nxx*nyy*fsize*nchannels) != nbytes:
# #                    raise ValueError('File size mismatch for %s. Fractional number of lines'%(fname))
# #            elif (nxx*nyy*fsize*nchannels) > nbytes:
# #                    raise ValueError('File size mismatch for %s. Number of bytes expected: %d'%(nbytes))
             

#         self.name = fname
#         self.width = nxx
#         self.length = nyy

#         ####List of memmap objects
#         acc = []

#         ####Create the memmap for the full file
#         nshape = nchannels*nyy*nxx
#         omap = np.memmap(fname, dtype=dataType, mode=mode, 
#                 shape = (nshape,))

#         if scheme.upper() == 'BIL':
#             nstrides = (nchannels*nxx*fsize, fsize)

#             for band in range(nchannels):
#                 ###Starting offset
#                 noffset = band*nxx

#                 ###Temporary view
#                 tmap = omap[noffset:]

#                 ####Trick it into creating a 2D array
#                 fmap = as_strided(tmap, shape=(nyy,nxx), strides=nstrides)

#                 ###Add to list of objects
#                 acc.append(fmap)

#         elif scheme.upper() == 'BSQ':
#             nstrides = (fsize, fsize)

#             for band in range(nchannels):
#                 ###Starting offset
#                 noffset = band*nxx*nyy

#                 ###Temporary view
#                 tmap = omap[noffset:noffset+nxx*nyy]

#                 ####Reshape into 2D array
#                 fmap = as_strided(tmap, shape=(nyy,nxx))

#                 ###Add to lits of objects
#                 acc.append(fmap)

#         elif scheme.upper() == 'BIP':
#             nstrides = (nchannels*nxx*fsize,nchannels*fsize)

#             for band in range(nchannels):
#                 ####Starting offset
#                 noffset = band

#                 ####Temporary view
#                 tmap = omap[noffset:]

#                 ####Trick it into interpreting ot as a 2D array
#                 fmap = as_strided(tmap, shape=(nyy,nxx), strides=nstrides)

#                 ####Add to the list of objects
#                 acc.append(fmap)

#         else:
#             raise ValueError('Unknown file scheme: %s for file %s'%(scheme,fname))

#         ######Assigning list of objects to self.bands
#         self.bands = acc

#     def flush(self):
#         '''
#         If mmap opened in write mode, would be useful to have flush functionality on old systems.
#         '''

#         self.bands[0].base.base.flush()

In [ ]:
# def gdal2isce_xml(fname):
#     """
#     Generate ISCE xml file from gdal supported file

#     Example: import isce
#              from applications.gdal2isce_xml import gdal2isce_xml
#              xml_file = gdal2isce_xml(fname+'.vrt')
#     """

#     # open the GDAL file and get typical data informationi
#     GDAL2ISCE_DATATYPE = {
#        1 : 'BYTE',
#        2 : 'uint16',
#        3 : 'SHORT',
#        4 : 'uint32',
#        5 : 'INT',
#        6 : 'FLOAT',
#        7 : 'DOUBLE',
#        10: 'CFLOAT',
#        11: 'complex128',
#     }
# #    GDAL2NUMPY_DATATYPE = {
# #       1 : np.uint8,
# #       2 : np.uint16,
# #       3 : np.int16,
# #       4 : np.uint32,
# #       5 : np.int32,
# #       6 : np.float32,
# #       7 : np.float64,
# #       10: np.complex64,
# #       11: np.complex128,
# #     }

#     # check if the input file is a vrt
#     fbase, fext = os.path.splitext(fname)
#     print(fext)
#     if fext == ".vrt":
#         outname = fbase
#     else:
#         outname = fname
#     print(outname)

#     # open the GDAL file and get typical ds information
#     ds =  gdal.Open(fname, gdal.GA_ReadOnly)
#     width = ds.RasterXSize
#     length = ds.RasterYSize
#     bands = ds.RasterCount
#     print("width:       " + "\t" + str(width))
#     print("length:      " + "\t" + str(length))
#     print("num of bands:" + "\t" + str(bands))

#     # getting the datatype information
#     raster = ds.GetRasterBand(1)
#     dataTypeGdal = raster.DataType

#     # user look-up dictionary from gdal to isce format
#     dataType= GDAL2ISCE_DATATYPE[dataTypeGdal]
#     print("dataType: " + "\t" + str(dataType))

#     # transformation contains gridcorners (lines/pixels or lonlat and the spacing 1/-1 or deltalon/deltalat)
#     transform = ds.GetGeoTransform()
#     # if a complex data type, then create complex image
#     # if a real data type, then create a regular image

#     img = isceobj.createImage()
#     img.setFilename(os.path.abspath(outname))
#     img.setWidth(width)
#     img.setLength(length)
#     img.setAccessMode('READ')
#     img.bands = bands
#     img.dataType = dataType

#     # interleave
#     md = ds.GetMetadata('IMAGE_STRUCTURE')
#     sch = md.get('INTERLEAVE', None)
#     if sch == 'LINE':
#         img.scheme = 'BIL'
#     elif sch == 'PIXEL':
#         img.scheme = 'BIP'
#     elif sch == 'BAND':
#         img.scheme = 'BSQ'
#     else:
#         print('Unrecognized interleaving scheme, {}'.format(sch))
#         print('Assuming default, BIP')
#         img.scheme = 'BIP'

#     img.firstLongitude = transform[0]
#     img.firstLatitude = transform[3] 
#     img.deltaLatitude = transform[5] 
#     img.deltaLongitude = transform[1]

#     xml_file = outname + ".xml"
#     img.dump(xml_file)

#     return xml_file


# Run geogrid and autoRIFT on all images in a folder

In [ ]:
import pandas as pd

s2path = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel2/' # input S2 images
LS8path = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/LS8images/useable_images/' # input LS8 images
PSpath = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/Planet_test/Planet_test_all/'
S1path = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel1/'
# PSpath = '/Volumes/SGlacier/surge_projects/Working_clipped_2019/'
# boxpath = '/Users/surging/Documents/TG/BoxTurner/BoxTurner_UTM_07.shp' # the shapefile for Turner
# autoriftpath = '/Users/surging/Documents/TG/autoRIFT/' # path to the autorift scripts
# vmap_path = '/Users/surging/Documents/TG/vmap_test/' # output velocity map folder
# basepath = '/Users/surging/Documents/TG/optical-offset-tracking/' # path where this script is located

######### Set minimum and maximum time separation and the platform (L8, S2, PS, S1) ###############
platform = 'L8'
startdate = '20130101' # inclusive start date
enddate = '20160101' # inclusive end date
min_dt = 3 # minimum time separation between images
max_dt = 30 # maximum time separation between images
###############################################################################################

In [ ]:
# assign the folder path based on the platform
if platform == 'S2': # sentinel-2
    path = s2path
    ext = '_clipped.tif' # image filename extension
    img_type = 'OPT'
    date_split_idx = 2 # split filename by underscore, index corresponds to image date
elif platform == 'L8': # landsat 8
    path = LS8path
    ext = '.TIF'
    img_type = 'OPT'
    date_split_idx = 3
elif platform == 'PS': # PlanetScope
    path = PSpath
    ext = '_clipped.tif' 
    img_type = 'OPT'
    date_split_idx = 1
elif platform == 'S1': # sentinel-1
    path = S1path
    ext = '.zip' 
    img_type = 'SAR'
    date_split_idx = 5
else:
    print('Platform', platform, 'not recognized. Options are "S2", "L8", "PS", and "S1"')

# record all possible images and their dates
dates = []; files = []  
for file in os.listdir(path):
    if file.endswith(ext):
        date = file.split('_')[date_split_idx] # grab the date from the filename
        if platform == 'S1': # S1 filenames need another split
            date = date[:8]
        dates.append(date); files.append(file) # store the date and filename
files_df = pd.DataFrame(list(zip(files,dates)),columns=['filename','date'])
files_df = files_df.sort_values(by='date',ignore_index=True) # sort the dataframe by ascending date
files_df

In [ ]:
# filter out those before and afte the input start and end date
files_df = files_df[(files_df.date >= startdate) & (files_df.date <= enddate)]
files_df = files_df.reset_index(drop=True) # reset index for searching
files_df

In [ ]:
########## PARAMETERS FOR GEOGRID #################################################### 
out_path = '/Users/jukesliu/Documents/TURNER/DATA/VELOCITY_MAPS/output_AutoRIFT/' # output file path
dem = dempath+dem_outfile # path to the resampled DEM produced in the previous step (outfile name)
#             MINCHIPSIZE = CHIPSIZE_M # smallest chip size allowed in image horizontal direction (in m)
#             print(MINCHIPSIZE)
NO_DATA_VAL = -32767 # no data value in the output products
temp_dir = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel1/coreg_files/'

# PARAMETERS FOR AUTORIFT CHOSEN
# options are HPS (high pass), WAL (wallis), SOB (sobel), DB (logarithmic operator)
FILTER = 'HPS'
WALLISFILTERWIDTH = 32 # only for wallis filter, must be a power of 2

# Sampling:
SPARSE_SEARCH_SAMPLE_RATE = 16 # how many samples to skip to speed up processing
OVERSAMPLE_RATIO = 0 # enter in a constant scalar or 0 for default parameters
print('Filters and autoRIFT sample rates chosen.')
############################################################################################

In [ ]:
######### ENTER DEM INFO, AND REFERENCE VELOCITY INFO ########### 
# enter in the path to your best DEM over the region
dempath = '/Users/jukesliu/Documents/TURNER/DATA/ICE_THICKNESS/surface/DEMs_previous/'
demname = 'IfSAR_5m_DSM_clipped.tif'

# path to the reference files for geogrid (vx, vy, ssm)
refvpath = '/Users/jukesliu/Documents/TURNER/DATA/VELOCITY_MAPS/forAutoRIFT/' 
vx_fname = 'vx_cropped.tif' # name of reference vx file
vy_fname = 'vy_cropped.tif' # name of reference vy file

sr_scaling = 16 # multiply by vx and vy to generate search range limits
############################################################################

In [ ]:
# try multiple chip sizes and sats
for platform in ['S2','L8']:
    # assign the folder path based on the platform
    if platform == 'S2': # sentinel-2
        path = s2path
        ext = '_clipped.tif' # image filename extension
        img_type = 'OPT'
        date_split_idx = 2 # split filename by underscore, index corresponds to image date
    elif platform == 'L8': # landsat 8
        path = LS8path
        ext = '.TIF'
        img_type = 'OPT'
        date_split_idx = 3
    elif platform == 'PS': # PlanetScope
        path = PSpath
        ext = '_clipped.tif' 
        img_type = 'OPT'
        date_split_idx = 1
    elif platform == 'S1': # sentinel-1
        path = S1path
        ext = '.zip' 
        img_type = 'SAR'
        date_split_idx = 5
    else:
        print('Platform', platform, 'not recognized. Options are "S2", "L8", "PS", and "S1"')

    # record all possible images and their dates
    dates = []; files = []  
    for file in os.listdir(path):
        if file.endswith(ext):
            date = file.split('_')[date_split_idx] # grab the date from the filename
            if platform == 'S1': # S1 filenames need another split
                date = date[:8]
            dates.append(date); files.append(file) # store the date and filename
    files_df = pd.DataFrame(list(zip(files,dates)),columns=['filename','date'])
    files_df = files_df.sort_values(by='date',ignore_index=True) # sort the dataframe by ascending date
    files_df = files_df[(files_df.date >= startdate) & (files_df.date <= enddate)]
    files_df = files_df.reset_index(drop=True) # reset index for searching
    
    for min_chipsize in [100,160,200,300]: # MULTIPLE CHIP SIZES
        MINCHIPSIZE = min_chipsize
        # generate geogrid inputs and grab outfilenames
        [dem_outfile, dhdx_outfile, dhdy_outfile, 
         vx_outfile, vy_outfile, srx_outfile, 
         sry_outfile, ssm_outfile, tg_outfile] = generate_geogrid_inputs(min_chipsize, dempath, demname, 
                                                                         refvpath, vx_fname, vy_fname, 
                                                                         sr_scaling)
        #######################################################################
        # # CHOOSE OPTIONAL INPUTS: (set as '' to leave blank)
        # # 1) surface slope:
        #   dhdx = ''; dhdy = ''  
        dhdx = dempath+dhdx_outfile; dhdy = dempath+dhdy_outfile
        
        # # 2) reference velocity:
        vx = ''; vy = ''    
#         vx = refvpath+vx_outfile; vy = refvpath+vy_outfile
        
        # # 3) chip sizes:
        csminx = ''; csminy = ''
        csmaxx = ''; csmaxy = ''  
        #             csminx = refvpath+csminx_fname; csminy = refvpath+csminy_fname
        #             csmaxx = refvpath+csmaxx_fname; csmaxy = refvpath+csmaxy_fname
        
        # # 4) stable surface mask:
        ssm = ''
        #             ssm = refvpath+ssm_outfile # stable surface mask 
        
        # # 5) search range limit:
        srx = ''; sry = '' # for best results, never input search range limit
        #######################################################################
        
        for rownum in range(0,len(files_df)-1):
            if rownum == 0: # for the first row, idx1 = 0 and idx2 = 1
                idx1 = rownum
            idx2 = idx1+1 # reset index 2 as the next item 

            if idx1 < len(files_df) and idx2 < len(files_df): # don't surpass the end of the data
                # identify the successive image pairs:
                m = files_df.loc[idx1,'filename']; s = files_df.loc[idx2, 'filename']

                # grab the two dates and convert to datetime objects
                d1s = m.split('_')[date_split_idx]; d2s = s.split('_')[date_split_idx]
                if platform == 'S1': # S1 filenames need another split
                    d1s = d1s[:8]; d2s = d2s[:8]
                
                # Skip if output already exists for this combination of parameters
#                 if os.path.exists(out_path+'offset_'+d1s+'_'+d2s+'_'+str(min_chipsize)+'m_'+platform+'.tif'):
#                     print('offset_'+d1s+'_'+d2s+'_'+str(min_chipsize)+'m_'+platform+'.tif already exists.')
#                 else:
                if True:
                    # calculate time separation
                    d1 = datetime.datetime.strptime(d1s, '%Y%m%d'); d2 = datetime.datetime.strptime(d2s, '%Y%m%d')
                    dt = d2-d1; dt = int(dt.days)

                    # evaluate if dates are within the appropriate time separation
                    while (dt < min_dt or dt > max_dt): # dt is outside of appropriate time separation
                        idx2 += 1 # increment index 2

                        if idx2 > len(files_df)-1:
                            print('No image pair with correct time separation found. Skip to next.')
                            idx2 = idx1+1 # reset index 2 as the next item 
                            m = None; s = None
                            break

                        # Keep looking for the correct time separation:
                        m = files_df.loc[idx1,'filename']; s = files_df.loc[idx2, 'filename']

                        # grab the two dates and convert to datetime objects
                        d1s = m.split('_')[date_split_idx]; d2s = s.split('_')[date_split_idx]
                        if platform == 'S1': # S1 filenames need another split
                            d1s = d1s[:8]; d2s = d2s[:8]
                        d1 = datetime.datetime.strptime(d1s, '%Y%m%d'); d2 = datetime.datetime.strptime(d2s, '%Y%m%d')
                        dt = d2-d1; dt = int(dt.days)

                    # run geogrid and autoRIFT
                    if m is not None and s is not None:
                        print(m,s)
                        indir_m = path+m; indir_s = path+s # path to the two images
                        dem = dempath+'IfSAR_'+str(MINCHIPSIZE)+'m_DSM_clipped.tif'
                        dhdx = dempath+'IfSAR_'+str(MINCHIPSIZE)+'m_DSM_clipped_dhdx.tif'
                        dhdy = dempath+'IfSAR_'+str(MINCHIPSIZE)+'m_DSM_clipped_dhdy.tif'
                        ###################### RUN GEOGRID ################################
                        run_geogrid_inhouse(out_path, img_type, indir_m, indir_s, MINCHIPSIZE, NO_DATA_VAL, dem, # required inputs
                                            dhdx, dhdy, vx, vy, srx, sry, csminx, csminy, csmaxx, csmaxy, ssm, # optional inputs
                                            temp_dir)

                        ##################### PREP AUTORIFT ##############################
                        gp = out_path # identify files produced from geogrid
                        # remove all empty grids
                        for grid in os.listdir(gp): 
                            if grid.startswith('window') and grid.endswith('.tif'):
                                reader = rio.open(gp+grid) # read dataset
                                data_found = False 
                                for band in range(1,reader.count+1):
                                    testband = reader.read(band) # read in the band
                                    if np.count_nonzero(testband[testband != NO_DATA_VAL]) > 0:
                                        data_found = True
                                if not data_found:
                                    print(grid, 'has no data. Removed.')
                                    os.remove(gp+grid)

                        # fill in AutoRIFT parameters using the files
                        mpflag = 0 # leave multiprocessing off

                        # GRID LOCATION (required) from window_location.tif
                        grid_location = rio.open(gp+'window_location.tif')
                        xGrid = grid_location.read(1) # 1st band in window location
                        yGrid = grid_location.read(2) # 2nd band in window location

                        # optional parameters (default None or zero until filled)
                        init_offset = None; search_range = None
                        chip_size_min = None; chip_size_max = None
                        offset2vx = None; offset2vy = None; stable_surface_mask = None
                        Dx0 = None; Dy0 = None; CSMINx0 = None
                        SRx0 = None; SRy0 = None;
                        CSMAXx0 = None; CSMAXy0 = None; SSM = None
                        noDataMask = np.zeros(xGrid.shape).astype(int)

                        if os.path.exists(gp+'window_offset.tif'): # Dx0 and Dy0 from window_offset.tif
                            init_offset = rio.open(gp+'window_offset.tif')
                            Dx0 = init_offset.read(1); Dy0 = init_offset.read(2)
                        if os.path.exists(gp+'window_search_range.tif'): # SRx0 and SRy0 from window_search_range.tif
                            search_range = rio.open(gp+'window_search_range.tif')
                            SRx0 = search_range.read(1); SRy0 = search_range.read(2)
                        if os.path.exists(gp+'window_chip_size_min.tif'): # CSMINx0 and CSMINy0 from window_chip_size_min.tif
                            chip_size_min = rio.open(gp+'window_chip_size_min.tif')
                            CSMINx0 = chip_size_min.read(1); CSMINy0 = chip_size_min.read(2)
                        if os.path.exists(gp+'window_chip_size_max.tif'): # CSMAXx0 and CSMAXy0 from window_chip_size_max.tif
                            chip_size_max = rio.open(gp+'window_chip_size_max.tif')
                            CSMAXx0 = chip_size_max.read(1); CSMAXy0 = chip_size_max.read(2)
                        if os.path.exists(gp+'window_rdr_off2vel_x_vec.tif'): # offset2vx from window_rdr_off2vel_x_vec.tif
                            offset2vx = gp+'window_rdr_off2vel_x_vec.tif' # path to be read in with GDAL
                        if os.path.exists(gp+'window_rdr_off2vel_y_vec.tif'): # offset2vy from window_rdr_off2vel_y_vec.tif
                            offset2vy = gp+'window_rdr_off2vel_y_vec.tif' 
                        if os.path.exists(gp+'window_stable_surface_mask.tif'): # noDataMask from window_stable_surface_mask.tif
                            stable_surface_mask = rio.open(gp+'window_stable_surface_mask.tif')
                            noDataMask = stable_surface_mask.read(1)

                        # other parameters
                        nodata = NO_DATA_VAL # use same as in previous steps
                        geogrid_run_info=None
                        print('AutoRIFT parameters loaded.')

                        ##################### RUN AUTORIFT ##############################
                        # run autoRIFT with function
                        run_autoRIFT_inhouse(out_path, img_type, mpflag, xGrid, yGrid, # required parameters
                                                 FILTER, WALLISFILTERWIDTH, SPARSE_SEARCH_SAMPLE_RATE, 
                                                 OVERSAMPLE_RATIO, MINCHIPSIZE,
                                                 Dx0, Dy0, CSMINx0, SRx0, SRy0, CSMAXx0, CSMAXy0, SSM, # optional parameters
                                                 noDataMask, nodata, geogrid_run_info)

                idx1 = idx2 # set the second image index as the new first image index
                if idx2 >= len(files_df)-1:
                    print('Finished searching.')
                    break
